## Entrenando  traductor español-ayuuk-español

In [2]:
%load_ext tensorboard

In [18]:
import os
import codecs

In [19]:
nvoca=4000

In [6]:
"""
project_name='mir'
source_language="es"
src=source_language
target_language="mir"
tgt=target_language
name=f"{project_name}.{source_language}{target_language}"
data_path = os.path.join("data-sin",name )

print("Nombre de proyecto     :", name)
print("Directorio de proyecto :", data_path)
"""

Nombre de proyecto     : mir.esmir
Directorio de proyecto : data-sin/mir.esmir


In [20]:
project_name='mir'
source_language="mir"
src=source_language
target_language="es"
tgt=target_language
name=f"{project_name}.{source_language}{target_language}"
data_path = os.path.join("/","data","joeynmt",name )

print("Nombre de proyecto     :", name)
print("Directorio de proyecto :", data_path)

Nombre de proyecto     : mir.mires
Directorio de proyecto : /data/joeynmt/mir.mires


In [21]:
! mkdir -p {data_path}
! mkdir -p {data_path}/models

## Crea vocabulario

Extrae el vocabulario subwords BPE del train

In [22]:
! /ve/bin/subword-nmt learn-joint-bpe-and-vocab --input /data/train.{project_name}.{src} /data/train.{project_name}.{tgt} -s {nvoca} -o {data_path}/bpe.codes.{nvoca} --write-vocabulary {data_path}/vocab.{src} {data_path}/vocab.{tgt}

In [23]:
! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{src} < /data/train.{project_name}.{src} > {data_path}/train.bpe.{src}
! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{tgt} < /data/train.{project_name}.{tgt} > {data_path}/train.bpe.{tgt}

! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{src} < /data/dev.{project_name}.{src} > {data_path}/dev.bpe.{src}
! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{tgt} < /data/dev.{project_name}.{tgt} > {data_path}/dev.bpe.{tgt}
! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{src} < /data/test.{project_name}.{src} > {data_path}/test.bpe.{src}
! /ve/bin/subword-nmt apply-bpe -c {data_path}/bpe.codes.{nvoca} --vocabulary {data_path}/vocab.{tgt} < /data/test.{project_name}.{tgt} > {data_path}/test.bpe.{tgt}

In [24]:
! chmod 777 joeynmt/scripts/build_vocab.py
! /ve/bin/python3 joeynmt/scripts/build_vocab.py {data_path}/train.bpe.{src} {data_path}/train.bpe.{tgt} --output_path {data_path}/vocab.txt

In [25]:
print("Contenido de directorio",data_path)
!ls {data_path}

Contenido de directorio /data/joeynmt/mir.mires
bpe.codes.4000	dev.bpe.mir  test.bpe.es   train.bpe.es   vocab.es   vocab.txt
dev.bpe.es	models	     test.bpe.mir  train.bpe.mir  vocab.mir


## Activando GPU

In [11]:
%env CUDA_DEVICE_ORDER="PCI_BUS_ID"
%env CUDA_VISIBLE_DEVICES="5,6"
! nvidia-smi

env: CUDA_DEVICE_ORDER="PCI_BUS_ID"
env: CUDA_VISIBLE_DEVICES="5,6"
Sat Mar 20 07:58:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:15:00.0 Off |                    0 |
| N/A   37C    P0    62W / 300W |  32270MiB / 32510MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:16:00.0 Off |                    0 |
| N/A   35C    P0    44W / 300W |     12MiB / 32510MiB |      0%      Default |
+-------------------

# ES-MIR RANDOM BPE 4000 LEN 70 CONF A 250 EPOCHS

In [12]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/es-mir-random-4000-mc701/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 250                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/es-mir-random-4000-mc701"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [13]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-20 08:03:51,146 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 08:03:51,207 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-20 08:03:51,529 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 08:05:06,400 - INFO - joeynmt.training - Epoch   1: total training loss 265.61
2021-03-20 08:05:06,402 - INFO - joeynmt.training - EPOCH 2
2021-03-20 08:06:10,049 - INFO - joeynmt.training - Epoch   2: total training loss 246.38
2021-03-20 08:06:10,049 - INFO - joeynmt.training - EPOCH 3
2021-03-20 08:07:12,160 - INFO - joeynmt.training - Epoch   3: total training loss 244.00
2021-03-20 08:07:12,162 - INFO - joeynmt.training - EPOCH 4
2021-03-20 08:08:18,423 - INFO - joeynmt.training - Epoch   4: total training loss 241.83
2021-03-20 08:08:18,424 - INFO - joeynmt.training - EP

2021-03-20 08:32:53,047 - INFO - joeynmt.training - Epoch  24: total training loss 183.43
2021-03-20 08:32:53,048 - INFO - joeynmt.training - EPOCH 25
2021-03-20 08:33:58,174 - INFO - joeynmt.training - Epoch  25: total training loss 181.02
2021-03-20 08:33:58,174 - INFO - joeynmt.training - EPOCH 26
2021-03-20 08:35:02,535 - INFO - joeynmt.training - Epoch  26: total training loss 179.50
2021-03-20 08:35:02,535 - INFO - joeynmt.training - EPOCH 27
2021-03-20 08:36:09,410 - INFO - joeynmt.training - Epoch  27: total training loss 176.52
2021-03-20 08:36:09,411 - INFO - joeynmt.training - EPOCH 28
2021-03-20 08:37:10,535 - INFO - joeynmt.training - Epoch  28: total training loss 174.90
2021-03-20 08:37:10,535 - INFO - joeynmt.training - EPOCH 29
2021-03-20 08:38:17,106 - INFO - joeynmt.training - Epoch  29: total training loss 172.15
2021-03-20 08:38:17,107 - INFO - joeynmt.training - EPOCH 30
2021-03-20 08:39:18,253 - INFO - joeynmt.training - Epoch  30: total training loss 170.57
2021

2021-03-20 09:02:04,869 - INFO - joeynmt.training - Epoch  48: total training loss 134.48
2021-03-20 09:02:04,869 - INFO - joeynmt.training - EPOCH 49
2021-03-20 09:03:13,087 - INFO - joeynmt.training - Epoch  49: total training loss 133.05
2021-03-20 09:03:13,087 - INFO - joeynmt.training - EPOCH 50
2021-03-20 09:04:16,699 - INFO - joeynmt.training - Epoch  50: total training loss 130.92
2021-03-20 09:04:16,700 - INFO - joeynmt.training - EPOCH 51
2021-03-20 09:05:25,334 - INFO - joeynmt.training - Epoch  51: total training loss 129.20
2021-03-20 09:05:25,334 - INFO - joeynmt.training - EPOCH 52
2021-03-20 09:06:32,207 - INFO - joeynmt.training - Epoch  52: total training loss 127.84
2021-03-20 09:06:32,207 - INFO - joeynmt.training - EPOCH 53
2021-03-20 09:07:36,653 - INFO - joeynmt.training - Epoch  53: total training loss 126.03
2021-03-20 09:07:36,653 - INFO - joeynmt.training - EPOCH 54
2021-03-20 09:08:37,486 - INFO - joeynmt.training - Epoch  54: total training loss 125.24
2021

2021-03-20 09:32:44,385 - INFO - joeynmt.training - Epoch  73: total training loss 101.83
2021-03-20 09:32:44,385 - INFO - joeynmt.training - EPOCH 74
2021-03-20 09:33:54,368 - INFO - joeynmt.training - Epoch  74: total training loss 100.14
2021-03-20 09:33:54,368 - INFO - joeynmt.training - EPOCH 75
2021-03-20 09:35:06,934 - INFO - joeynmt.training - Epoch  75: total training loss 99.09
2021-03-20 09:35:06,935 - INFO - joeynmt.training - EPOCH 76
2021-03-20 09:36:11,199 - INFO - joeynmt.training - Epoch  76: total training loss 97.67
2021-03-20 09:36:11,199 - INFO - joeynmt.training - EPOCH 77
2021-03-20 09:37:25,082 - INFO - joeynmt.training - Epoch  77: total training loss 96.87
2021-03-20 09:37:25,082 - INFO - joeynmt.training - EPOCH 78
2021-03-20 09:38:36,620 - INFO - joeynmt.training - Epoch  78: total training loss 95.89
2021-03-20 09:38:36,620 - INFO - joeynmt.training - EPOCH 79
2021-03-20 09:39:44,497 - INFO - joeynmt.training - Epoch  79: total training loss 95.11
2021-03-2

...

2021-03-20 10:40:26,518 - INFO - joeynmt.training - Example #0
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Source:     promover el uso de las lenguas
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Reference:  yajtuyo'opy je ajxy jeky aambyë
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Hypothesis: majk kilo
2021-03-20 10:40:26,518 - INFO - joeynmt.training - Example #1
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Source:     pues si david le llama señor, ¿cómo es su hijo?
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Reference:  ¿nebyëk tyijyë david ja cristo jeꞌe duꞌun yꞌapꞌaty yꞌokꞌaty, pë david ja cristo jëduꞌun amdsoo yajxëꞌajp mëj windsën?
2021-03-20 10:40:26,518 - INFO - joeynmt.training - 	Hypothesis: a pë ja dios ja aatsy të wyinkpëkyëbë juxytsyëgiꞌx jak moꞌowaambyë, ¿nebyëk tyijy jeꞌe ja yaꞌadyëjk ja yaꞌay ënaꞌk xynyëmaꞌawaambë?
2021-03-20 10:40:26,518 - INFO - joeynmt.training - Example #2
2021-03-20 10:40:26,519 - INFO - joeynmt.training 

2021-03-20 11:07:47,815 - INFO - joeynmt.training - Validation result (greedy) at epoch 152, step     6500: bleu:   6.92, loss: 91387.6641, ppl:  52.1231, duration: 85.1623s
2021-03-20 11:08:43,597 - INFO - joeynmt.training - Epoch 152: total training loss 54.93
2021-03-20 11:08:43,598 - INFO - joeynmt.training - EPOCH 153
2021-03-20 11:09:48,155 - INFO - joeynmt.training - Epoch 153: total training loss 55.03
2021-03-20 11:09:48,155 - INFO - joeynmt.training - EPOCH 154
2021-03-20 11:10:59,126 - INFO - joeynmt.training - Epoch 154: total training loss 54.48
2021-03-20 11:10:59,127 - INFO - joeynmt.training - EPOCH 155
2021-03-20 11:12:01,876 - INFO - joeynmt.training - Epoch 155: total training loss 54.25
2021-03-20 11:12:01,876 - INFO - joeynmt.training - EPOCH 156
2021-03-20 11:13:11,377 - INFO - joeynmt.training - Epoch 156: total training loss 54.15
2021-03-20 11:13:11,379 - INFO - joeynmt.training - EPOCH 157
2021-03-20 11:14:12,536 - INFO - joeynmt.training - Epoch 157: total tr

2021-03-20 11:41:57,263 - INFO - joeynmt.training - Epoch 179: total training loss 47.21
2021-03-20 11:41:57,264 - INFO - joeynmt.training - EPOCH 180
2021-03-20 11:43:02,183 - INFO - joeynmt.training - Epoch 180: total training loss 47.22
2021-03-20 11:43:02,183 - INFO - joeynmt.training - EPOCH 181
2021-03-20 11:44:15,555 - INFO - joeynmt.training - Epoch 181: total training loss 47.06
2021-03-20 11:44:15,556 - INFO - joeynmt.training - EPOCH 182
2021-03-20 11:45:28,552 - INFO - joeynmt.training - Epoch 182: total training loss 46.75
2021-03-20 11:45:28,552 - INFO - joeynmt.training - EPOCH 183
2021-03-20 11:46:37,332 - INFO - joeynmt.training - Epoch 183: total training loss 46.49
2021-03-20 11:46:37,332 - INFO - joeynmt.training - EPOCH 184
2021-03-20 11:47:51,640 - INFO - joeynmt.training - Epoch 184: total training loss 46.40
2021-03-20 11:47:51,641 - INFO - joeynmt.training - EPOCH 185
2021-03-20 11:49:04,199 - INFO - joeynmt.training - Epoch 185: total training loss 46.09
2021-

2021-03-20 12:19:08,237 - INFO - joeynmt.training - Epoch 208: total training loss 42.36
2021-03-20 12:19:08,237 - INFO - joeynmt.training - EPOCH 209
2021-03-20 12:20:21,087 - INFO - joeynmt.training - Epoch 209: total training loss 42.37
2021-03-20 12:20:21,087 - INFO - joeynmt.training - EPOCH 210
2021-03-20 12:20:42,426 - INFO - joeynmt.training - Epoch 210, Step:     9000, Batch Loss:     1.001794, Tokens per Sec:     1655, Lr: 0.000210
2021-03-20 12:22:02,589 - INFO - joeynmt.training - Example #0
2021-03-20 12:22:02,590 - INFO - joeynmt.training - 	Source:     promover el uso de las lenguas
2021-03-20 12:22:02,590 - INFO - joeynmt.training - 	Reference:  yajtuyo'opy je ajxy jeky aambyë
2021-03-20 12:22:02,590 - INFO - joeynmt.training - 	Hypothesis: majk kilo jyemdsë
2021-03-20 12:22:02,590 - INFO - joeynmt.training - Example #1
2021-03-20 12:22:02,590 - INFO - joeynmt.training - 	Source:     pues si david le llama señor, ¿cómo es su hijo?
2021-03-20 12:22:02,590 - INFO - joeynm

2021-03-20 12:54:26,283 - INFO - joeynmt.training - Epoch 233: total training loss 38.85
2021-03-20 12:54:26,283 - INFO - joeynmt.training - EPOCH 234
2021-03-20 12:55:30,345 - INFO - joeynmt.training - Epoch 234: total training loss 38.26
2021-03-20 12:55:30,346 - INFO - joeynmt.training - EPOCH 235
2021-03-20 12:56:42,512 - INFO - joeynmt.training - Epoch 235: total training loss 38.05
2021-03-20 12:56:42,512 - INFO - joeynmt.training - EPOCH 236
2021-03-20 12:57:52,547 - INFO - joeynmt.training - Epoch 236: total training loss 37.88
2021-03-20 12:57:52,548 - INFO - joeynmt.training - EPOCH 237
2021-03-20 12:59:04,279 - INFO - joeynmt.training - Epoch 237: total training loss 37.74
2021-03-20 12:59:04,279 - INFO - joeynmt.training - EPOCH 238
2021-03-20 13:00:14,650 - INFO - joeynmt.training - Epoch 238: total training loss 37.61
2021-03-20 13:00:14,651 - INFO - joeynmt.training - EPOCH 239
2021-03-20 13:01:24,665 - INFO - joeynmt.training - Epoch 239: total training loss 37.55
2021-

In [17]:
# prueba traductor es-mir con random
! cd joeynmt; /ve/bin/python3 -m joeynmt translate models/es-mir-random-4000-mc701/config.yaml < models/es-mir-random-4000-mc701/frases-es_prueba.txt

2021-03-20 13:48:20,112 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 relea

# MIR-ES RANDOM BPE 4000 LEN 70 CONF A 250 EPOCHS

In [26]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/mir-es-random-4000-mc702/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 250                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/mir-es-random-4000-mc702"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [27]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-20 14:00:01,446 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 14:00:01,532 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-20 14:00:01,858 - INFO - joeynmt.data - building v

2021-03-20 14:01:17,585 - INFO - joeynmt.training - Epoch   1: total training loss 260.25
2021-03-20 14:01:17,585 - INFO - joeynmt.training - EPOCH 2
2021-03-20 14:02:28,648 - INFO - joeynmt.training - Epoch   2: total training loss 236.67
2021-03-20 14:02:28,649 - INFO - joeynmt.training - EPOCH 3
2021-03-20 14:03:40,419 - INFO - joeynmt.training - Epoch   3: total training loss 234.39
2021-03-20 14:03:40,420 - INFO - joeynmt.training - EPOCH 4
2021-03-20 14:04:49,651 - INFO - joeynmt.training - Epoch   4: total training loss 232.04
2021-03-20 14:04:49,651 - INFO - joeynmt.training - EPOCH 5
2021-03-20 14:05:59,925 - INFO - joeynmt.training - Epoch   5: total training loss 229.18
2021-03-20 14:05:59,926 - INFO - joeynmt.training - EPOCH 6
2021-03-20 14:07:02,603 - INFO - joeynmt.training - Epoch   6: total training loss 226.51
2021-03-20 14:07:02,603 - INFO - joeynmt.training - EPOCH 7
2021-03-20 14:08:12,427 - INFO - joeynmt.training - Epoch   7: total training loss 223.77
2021-03-20

2021-03-20 14:31:12,775 - INFO - joeynmt.training - Epoch  24: total training loss 177.44
2021-03-20 14:31:12,775 - INFO - joeynmt.training - EPOCH 25
2021-03-20 14:32:20,791 - INFO - joeynmt.training - Epoch  25: total training loss 175.81
2021-03-20 14:32:20,793 - INFO - joeynmt.training - EPOCH 26
2021-03-20 14:33:31,930 - INFO - joeynmt.training - Epoch  26: total training loss 173.32
2021-03-20 14:33:31,931 - INFO - joeynmt.training - EPOCH 27
2021-03-20 14:34:45,325 - INFO - joeynmt.training - Epoch  27: total training loss 171.21
2021-03-20 14:34:45,325 - INFO - joeynmt.training - EPOCH 28
2021-03-20 14:35:55,933 - INFO - joeynmt.training - Epoch  28: total training loss 168.89
2021-03-20 14:35:55,933 - INFO - joeynmt.training - EPOCH 29
2021-03-20 14:37:08,464 - INFO - joeynmt.training - Epoch  29: total training loss 166.48
2021-03-20 14:37:08,465 - INFO - joeynmt.training - EPOCH 30
2021-03-20 14:38:24,652 - INFO - joeynmt.training - Epoch  30: total training loss 164.81
2021

2021-03-20 15:04:39,246 - INFO - joeynmt.training - Epoch  49: total training loss 130.00
2021-03-20 15:04:39,247 - INFO - joeynmt.training - EPOCH 50
2021-03-20 15:05:52,551 - INFO - joeynmt.training - Epoch  50: total training loss 128.95
2021-03-20 15:05:52,551 - INFO - joeynmt.training - EPOCH 51
2021-03-20 15:06:59,108 - INFO - joeynmt.training - Epoch  51: total training loss 126.90
2021-03-20 15:06:59,109 - INFO - joeynmt.training - EPOCH 52
2021-03-20 15:08:10,767 - INFO - joeynmt.training - Epoch  52: total training loss 125.76
2021-03-20 15:08:10,767 - INFO - joeynmt.training - EPOCH 53
2021-03-20 15:09:20,983 - INFO - joeynmt.training - Epoch  53: total training loss 124.03
2021-03-20 15:09:20,984 - INFO - joeynmt.training - EPOCH 54
2021-03-20 15:10:32,178 - INFO - joeynmt.training - Epoch  54: total training loss 123.75
2021-03-20 15:10:32,178 - INFO - joeynmt.training - EPOCH 55
2021-03-20 15:11:42,080 - INFO - joeynmt.training - Epoch  55: total training loss 121.43
2021

2021-03-20 15:39:34,281 - INFO - joeynmt.training - Epoch  75: total training loss 96.79
2021-03-20 15:39:34,282 - INFO - joeynmt.training - EPOCH 76
2021-03-20 15:40:48,556 - INFO - joeynmt.training - Epoch  76: total training loss 95.51
2021-03-20 15:40:48,557 - INFO - joeynmt.training - EPOCH 77
2021-03-20 15:42:02,616 - INFO - joeynmt.training - Epoch  77: total training loss 94.52
2021-03-20 15:42:02,617 - INFO - joeynmt.training - EPOCH 78
2021-03-20 15:43:20,089 - INFO - joeynmt.training - Epoch  78: total training loss 93.76
2021-03-20 15:43:20,090 - INFO - joeynmt.training - EPOCH 79
2021-03-20 15:44:37,065 - INFO - joeynmt.training - Epoch  79: total training loss 92.45
2021-03-20 15:44:37,065 - INFO - joeynmt.training - EPOCH 80
2021-03-20 15:45:49,476 - INFO - joeynmt.training - Epoch  80: total training loss 91.69
2021-03-20 15:45:49,477 - INFO - joeynmt.training - EPOCH 81
2021-03-20 15:47:08,485 - INFO - joeynmt.training - Epoch  81: total training loss 90.79
2021-03-20 

...

2021-03-20 16:55:25,599 - INFO - joeynmt.training - Epoch 128: total training loss 60.55
2021-03-20 16:55:25,601 - INFO - joeynmt.training - EPOCH 129
2021-03-20 16:56:34,920 - INFO - joeynmt.training - Epoch 129: total training loss 59.65
2021-03-20 16:56:34,921 - INFO - joeynmt.training - EPOCH 130
2021-03-20 16:57:41,364 - INFO - joeynmt.training - Epoch 130: total training loss 59.04
2021-03-20 16:57:41,364 - INFO - joeynmt.training - EPOCH 131
2021-03-20 16:58:49,460 - INFO - joeynmt.training - Epoch 131: total training loss 58.91
2021-03-20 16:58:49,461 - INFO - joeynmt.training - EPOCH 132
2021-03-20 16:59:57,875 - INFO - joeynmt.training - Epoch 132: total training loss 58.02
2021-03-20 16:59:57,876 - INFO - joeynmt.training - EPOCH 133
2021-03-20 17:01:07,558 - INFO - joeynmt.training - Epoch 133: total training loss 57.62
2021-03-20 17:01:07,558 - INFO - joeynmt.training - EPOCH 134
2021-03-20 17:02:15,930 - INFO - joeynmt.training - Epoch 134: total training loss 57.15
2021-

2021-03-20 17:31:25,784 - INFO - joeynmt.training - Epoch 157: total training loss 48.89
2021-03-20 17:31:25,785 - INFO - joeynmt.training - EPOCH 158
2021-03-20 17:32:36,954 - INFO - joeynmt.training - Epoch 158: total training loss 48.34
2021-03-20 17:32:36,954 - INFO - joeynmt.training - EPOCH 159
2021-03-20 17:33:47,015 - INFO - joeynmt.training - Epoch 159: total training loss 48.22
2021-03-20 17:33:47,015 - INFO - joeynmt.training - EPOCH 160
2021-03-20 17:35:00,482 - INFO - joeynmt.training - Epoch 160: total training loss 48.13
2021-03-20 17:35:00,483 - INFO - joeynmt.training - EPOCH 161
2021-03-20 17:36:10,511 - INFO - joeynmt.training - Epoch 161: total training loss 47.90
2021-03-20 17:36:10,511 - INFO - joeynmt.training - EPOCH 162
2021-03-20 17:37:21,927 - INFO - joeynmt.training - Epoch 162: total training loss 47.60
2021-03-20 17:37:21,928 - INFO - joeynmt.training - EPOCH 163
2021-03-20 17:38:20,714 - INFO - joeynmt.training - Epoch 163, Step:     7000, Batch Loss:    

2021-03-20 18:09:46,918 - INFO - joeynmt.training - Epoch 186: total training loss 42.28
2021-03-20 18:09:46,919 - INFO - joeynmt.training - EPOCH 187
2021-03-20 18:09:49,035 - INFO - joeynmt.training - Epoch 187, Step:     8000, Batch Loss:     0.661979, Tokens per Sec:     1335, Lr: 0.000210
2021-03-20 18:11:26,601 - INFO - joeynmt.training - Example #0
2021-03-20 18:11:26,601 - INFO - joeynmt.training - 	Source:     yajtuyo'opy je ajxy jeky aambyë
2021-03-20 18:11:26,601 - INFO - joeynmt.training - 	Reference:  promover el uso de las lenguas
2021-03-20 18:11:26,601 - INFO - joeynmt.training - 	Hypothesis: encantan los meisos
2021-03-20 18:11:26,601 - INFO - joeynmt.training - Example #1
2021-03-20 18:11:26,601 - INFO - joeynmt.training - 	Source:     ¿nebyëk tyijyë david ja cristo jeꞌe duꞌun yꞌapꞌaty yꞌokꞌaty, pë david ja cristo jëduꞌun amdsoo yajxëꞌajp mëj windsën?
2021-03-20 18:11:26,602 - INFO - joeynmt.training - 	Reference:  pues si david le llama señor, ¿cómo es su hijo?
2021-

2021-03-20 18:42:30,170 - INFO - joeynmt.training - Epoch 210: total training loss 37.97
2021-03-20 18:42:30,170 - INFO - joeynmt.training - EPOCH 211
2021-03-20 18:43:32,514 - INFO - joeynmt.training - Epoch 211: total training loss 37.37
2021-03-20 18:43:32,514 - INFO - joeynmt.training - EPOCH 212
2021-03-20 18:44:43,365 - INFO - joeynmt.training - Epoch 212: total training loss 37.11
2021-03-20 18:44:43,366 - INFO - joeynmt.training - EPOCH 213
2021-03-20 18:45:54,395 - INFO - joeynmt.training - Epoch 213: total training loss 36.95
2021-03-20 18:45:54,396 - INFO - joeynmt.training - EPOCH 214
2021-03-20 18:47:03,944 - INFO - joeynmt.training - Epoch 214: total training loss 36.50
2021-03-20 18:47:03,944 - INFO - joeynmt.training - EPOCH 215
2021-03-20 18:48:15,224 - INFO - joeynmt.training - Epoch 215: total training loss 36.44
2021-03-20 18:48:15,224 - INFO - joeynmt.training - EPOCH 216
2021-03-20 18:49:20,155 - INFO - joeynmt.training - Epoch 216: total training loss 36.54
2021-

2021-03-20 19:29:22,202 - INFO - joeynmt.training - Epoch 245: total training loss 33.25
2021-03-20 19:29:22,202 - INFO - joeynmt.training - EPOCH 246
2021-03-20 19:30:36,737 - INFO - joeynmt.training - Epoch 246: total training loss 33.18
2021-03-20 19:30:36,737 - INFO - joeynmt.training - EPOCH 247
2021-03-20 19:31:49,219 - INFO - joeynmt.training - Epoch 247: total training loss 32.86
2021-03-20 19:31:49,219 - INFO - joeynmt.training - EPOCH 248
2021-03-20 19:33:04,094 - INFO - joeynmt.training - Epoch 248: total training loss 33.04
2021-03-20 19:33:04,095 - INFO - joeynmt.training - EPOCH 249
2021-03-20 19:34:14,120 - INFO - joeynmt.training - Epoch 249: total training loss 32.91
2021-03-20 19:34:14,121 - INFO - joeynmt.training - EPOCH 250
2021-03-20 19:35:22,583 - INFO - joeynmt.training - Epoch 250: total training loss 32.76
2021-03-20 19:35:22,584 - INFO - joeynmt.training - Training ended after 250 epochs.
2021-03-20 19:35:22,584 - INFO - joeynmt.training - Best validation res

In [32]:
! cd joeynmt; /ve/bin/python3 -m joeynmt translate models/mir-es-random-4000-mc702/config.yaml < models/mir-es-random-4000-mc702/frases-mir_prueba_2.txt

2021-03-20 20:15:15,256 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 relea

## con BPE 4000

## colab 4 lomgitud 70

In [13]:

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "models/transformer_colab4/1.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 150                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/transformer_colab4"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [14]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-12 15:47:00,966 - INFO - root - Hello! This is Joey-NMT (version 1.2).
/ve/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-12 15:47:01,011 - INFO - joeynmt.data - Loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserW

2021-03-12 15:47:48,329 - INFO - joeynmt.training - Epoch   1: total training loss 267.04
2021-03-12 15:47:48,329 - INFO - joeynmt.training - EPOCH 2
2021-03-12 15:48:33,301 - INFO - joeynmt.training - Epoch   2: total training loss 247.41
2021-03-12 15:48:33,301 - INFO - joeynmt.training - EPOCH 3
2021-03-12 15:49:19,151 - INFO - joeynmt.training - Epoch   3: total training loss 244.99
2021-03-12 15:49:19,152 - INFO - joeynmt.training - EPOCH 4
2021-03-12 15:50:05,367 - INFO - joeynmt.training - Epoch   4: total training loss 243.00
2021-03-12 15:50:05,367 - INFO - joeynmt.training - EPOCH 5
2021-03-12 15:50:50,115 - INFO - joeynmt.training - Epoch   5: total training loss 239.82
2021-03-12 15:50:50,116 - INFO - joeynmt.training - EPOCH 6
2021-03-12 15:51:34,682 - INFO - joeynmt.training - Epoch   6: total training loss 236.50
2021-03-12 15:51:34,682 - INFO - joeynmt.training - EPOCH 7
2021-03-12 15:52:16,213 - INFO - joeynmt.training - Epoch   7: total training loss 233.15
2021-03-12

2021-03-12 16:06:30,066 - INFO - joeynmt.training - Epoch  24: total training loss 182.31
2021-03-12 16:06:30,066 - INFO - joeynmt.training - EPOCH 25
2021-03-12 16:07:11,669 - INFO - joeynmt.training - Epoch  25: total training loss 179.85
2021-03-12 16:07:11,669 - INFO - joeynmt.training - EPOCH 26
2021-03-12 16:07:52,672 - INFO - joeynmt.training - Epoch  26: total training loss 177.55
2021-03-12 16:07:52,672 - INFO - joeynmt.training - EPOCH 27
2021-03-12 16:08:34,403 - INFO - joeynmt.training - Epoch  27: total training loss 175.62
2021-03-12 16:08:34,404 - INFO - joeynmt.training - EPOCH 28
2021-03-12 16:09:15,930 - INFO - joeynmt.training - Epoch  28: total training loss 172.69
2021-03-12 16:09:15,930 - INFO - joeynmt.training - EPOCH 29
2021-03-12 16:09:57,441 - INFO - joeynmt.training - Epoch  29: total training loss 170.31
2021-03-12 16:09:57,441 - INFO - joeynmt.training - EPOCH 30
2021-03-12 16:10:38,412 - INFO - joeynmt.training - Epoch  30: total training loss 168.14
2021

2021-03-12 16:26:10,897 - INFO - joeynmt.training - Epoch  49: total training loss 131.49
2021-03-12 16:26:10,898 - INFO - joeynmt.training - EPOCH 50
2021-03-12 16:26:52,614 - INFO - joeynmt.training - Epoch  50: total training loss 130.76
2021-03-12 16:26:52,615 - INFO - joeynmt.training - EPOCH 51
2021-03-12 16:27:33,806 - INFO - joeynmt.training - Epoch  51: total training loss 128.82
2021-03-12 16:27:33,806 - INFO - joeynmt.training - EPOCH 52
2021-03-12 16:28:15,163 - INFO - joeynmt.training - Epoch  52: total training loss 127.10
2021-03-12 16:28:15,163 - INFO - joeynmt.training - EPOCH 53
2021-03-12 16:28:57,180 - INFO - joeynmt.training - Epoch  53: total training loss 125.56
2021-03-12 16:28:57,180 - INFO - joeynmt.training - EPOCH 54
2021-03-12 16:29:38,532 - INFO - joeynmt.training - Epoch  54: total training loss 124.23
2021-03-12 16:29:38,532 - INFO - joeynmt.training - EPOCH 55
2021-03-12 16:30:21,134 - INFO - joeynmt.training - Epoch  55: total training loss 123.30
2021

2021-03-12 16:44:27,413 - INFO - joeynmt.training - Epoch  72: total training loss 103.23
2021-03-12 16:44:27,413 - INFO - joeynmt.training - EPOCH 73
2021-03-12 16:45:07,831 - INFO - joeynmt.training - Epoch  73: total training loss 101.90
2021-03-12 16:45:07,832 - INFO - joeynmt.training - EPOCH 74
2021-03-12 16:45:48,591 - INFO - joeynmt.training - Epoch  74: total training loss 100.77
2021-03-12 16:45:48,593 - INFO - joeynmt.training - EPOCH 75
2021-03-12 16:46:30,392 - INFO - joeynmt.training - Epoch  75: total training loss 99.88
2021-03-12 16:46:30,393 - INFO - joeynmt.training - EPOCH 76
2021-03-12 16:47:11,219 - INFO - joeynmt.training - Epoch  76: total training loss 98.67
2021-03-12 16:47:11,219 - INFO - joeynmt.training - EPOCH 77
2021-03-12 16:47:52,660 - INFO - joeynmt.training - Epoch  77: total training loss 97.87
2021-03-12 16:47:52,660 - INFO - joeynmt.training - EPOCH 78
2021-03-12 16:48:33,324 - INFO - joeynmt.training - Epoch  78: total training loss 97.21
2021-03-

2021-03-12 17:02:24,726 - INFO - joeynmt.training - Epoch  95: total training loss 83.58
2021-03-12 17:02:24,726 - INFO - joeynmt.training - EPOCH 96
2021-03-12 17:03:06,158 - INFO - joeynmt.training - Epoch  96: total training loss 83.18
2021-03-12 17:03:06,160 - INFO - joeynmt.training - EPOCH 97
2021-03-12 17:03:49,067 - INFO - joeynmt.training - Epoch  97: total training loss 81.94
2021-03-12 17:03:49,067 - INFO - joeynmt.training - EPOCH 98
2021-03-12 17:04:30,292 - INFO - joeynmt.training - Epoch  98: total training loss 81.34
2021-03-12 17:04:30,292 - INFO - joeynmt.training - EPOCH 99
2021-03-12 17:05:11,629 - INFO - joeynmt.training - Epoch  99: total training loss 80.87
2021-03-12 17:05:11,629 - INFO - joeynmt.training - EPOCH 100
2021-03-12 17:05:53,547 - INFO - joeynmt.training - Epoch 100: total training loss 80.09
2021-03-12 17:05:53,549 - INFO - joeynmt.training - EPOCH 101
2021-03-12 17:06:34,473 - INFO - joeynmt.training - Epoch 101: total training loss 79.84
2021-03-1

2021-03-12 17:21:51,586 - INFO - joeynmt.training - Epoch 120: total training loss 70.20
2021-03-12 17:21:51,587 - INFO - joeynmt.training - EPOCH 121
2021-03-12 17:22:34,610 - INFO - joeynmt.training - Epoch 121: total training loss 69.25
2021-03-12 17:22:34,610 - INFO - joeynmt.training - EPOCH 122
2021-03-12 17:23:16,424 - INFO - joeynmt.training - Epoch 122: total training loss 68.15
2021-03-12 17:23:16,425 - INFO - joeynmt.training - EPOCH 123
2021-03-12 17:23:58,680 - INFO - joeynmt.training - Epoch 123: total training loss 67.57
2021-03-12 17:23:58,681 - INFO - joeynmt.training - EPOCH 124
2021-03-12 17:24:41,365 - INFO - joeynmt.training - Epoch 124: total training loss 67.22
2021-03-12 17:24:41,365 - INFO - joeynmt.training - EPOCH 125
2021-03-12 17:25:23,925 - INFO - joeynmt.training - Epoch 125: total training loss 67.00
2021-03-12 17:25:23,925 - INFO - joeynmt.training - EPOCH 126
2021-03-12 17:26:06,077 - INFO - joeynmt.training - Epoch 126: total training loss 66.64
2021-

2021-03-12 17:41:16,754 - INFO - joeynmt.training - Epoch 145: total training loss 58.78
2021-03-12 17:41:16,754 - INFO - joeynmt.training - EPOCH 146
2021-03-12 17:41:56,163 - INFO - joeynmt.training - Epoch 146: total training loss 58.78
2021-03-12 17:41:56,164 - INFO - joeynmt.training - EPOCH 147
2021-03-12 17:42:36,019 - INFO - joeynmt.training - Epoch 147: total training loss 58.34
2021-03-12 17:42:36,019 - INFO - joeynmt.training - EPOCH 148
2021-03-12 17:43:15,768 - INFO - joeynmt.training - Epoch 148: total training loss 57.99
2021-03-12 17:43:15,768 - INFO - joeynmt.training - EPOCH 149
2021-03-12 17:43:56,164 - INFO - joeynmt.training - Epoch 149: total training loss 57.50
2021-03-12 17:43:56,164 - INFO - joeynmt.training - EPOCH 150
2021-03-12 17:44:35,328 - INFO - joeynmt.training - Epoch 150: total training loss 57.35
2021-03-12 17:44:35,328 - INFO - joeynmt.training - Training ended after 150 epochs.
2021-03-12 17:44:35,328 - INFO - joeynmt.training - Best validation res

In [15]:
! cd joeynmt; /ve/bin/python3 -m joeynmt test configs/transformer_{name}.yaml

2021-03-12 17:48:55,604 - INFO - root - Hello! This is Joey-NMT (version 1.2).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-12 17:48:55,604 - INFO - joeynmt.data - Building vocabulary...
2021-03-12 17:48:55,758 - INFO - joeynmt.data - Loading dev data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
2021-03-12 17:48

## colab con 300 epocas

In [11]:

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{data_path}/models/sin-random1/1.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 300                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "{data_path}/models/sin-random1"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [12]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-13 00:48:15,463 - INFO - root - Hello! This is Joey-NMT (version 1.2).
/ve/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-13 00:48:15,508 - INFO - joeynmt.data - Loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserW

2021-03-13 00:49:05,869 - INFO - joeynmt.training - Epoch   1: total training loss 267.04
2021-03-13 00:49:05,870 - INFO - joeynmt.training - EPOCH 2
2021-03-13 00:49:45,420 - INFO - joeynmt.training - Epoch   2: total training loss 247.41
2021-03-13 00:49:45,420 - INFO - joeynmt.training - EPOCH 3
2021-03-13 00:50:25,519 - INFO - joeynmt.training - Epoch   3: total training loss 244.99
2021-03-13 00:50:25,519 - INFO - joeynmt.training - EPOCH 4
2021-03-13 00:51:04,862 - INFO - joeynmt.training - Epoch   4: total training loss 243.00
2021-03-13 00:51:04,862 - INFO - joeynmt.training - EPOCH 5
2021-03-13 00:51:43,759 - INFO - joeynmt.training - Epoch   5: total training loss 239.82
2021-03-13 00:51:43,759 - INFO - joeynmt.training - EPOCH 6
2021-03-13 00:52:23,648 - INFO - joeynmt.training - Epoch   6: total training loss 236.50
2021-03-13 00:52:23,649 - INFO - joeynmt.training - EPOCH 7
2021-03-13 00:53:02,926 - INFO - joeynmt.training - Epoch   7: total training loss 233.15
2021-03-13

2021-03-13 01:06:02,725 - INFO - joeynmt.training - Epoch  24: total training loss 182.31
2021-03-13 01:06:02,725 - INFO - joeynmt.training - EPOCH 25
2021-03-13 01:06:42,991 - INFO - joeynmt.training - Epoch  25: total training loss 179.85
2021-03-13 01:06:42,991 - INFO - joeynmt.training - EPOCH 26
2021-03-13 01:07:23,691 - INFO - joeynmt.training - Epoch  26: total training loss 177.55
2021-03-13 01:07:23,691 - INFO - joeynmt.training - EPOCH 27
2021-03-13 01:08:04,410 - INFO - joeynmt.training - Epoch  27: total training loss 175.62
2021-03-13 01:08:04,410 - INFO - joeynmt.training - EPOCH 28
2021-03-13 01:08:45,630 - INFO - joeynmt.training - Epoch  28: total training loss 172.69
2021-03-13 01:08:45,631 - INFO - joeynmt.training - EPOCH 29
2021-03-13 01:09:26,467 - INFO - joeynmt.training - Epoch  29: total training loss 170.31
2021-03-13 01:09:26,467 - INFO - joeynmt.training - EPOCH 30
2021-03-13 01:10:05,594 - INFO - joeynmt.training - Epoch  30: total training loss 168.14
2021

2021-03-13 01:24:10,954 - INFO - joeynmt.training - Epoch  49: total training loss 131.49
2021-03-13 01:24:10,955 - INFO - joeynmt.training - EPOCH 50
2021-03-13 01:24:50,536 - INFO - joeynmt.training - Epoch  50: total training loss 130.76
2021-03-13 01:24:50,536 - INFO - joeynmt.training - EPOCH 51
2021-03-13 01:25:29,498 - INFO - joeynmt.training - Epoch  51: total training loss 128.82
2021-03-13 01:25:29,499 - INFO - joeynmt.training - EPOCH 52
2021-03-13 01:26:07,437 - INFO - joeynmt.training - Epoch  52: total training loss 127.10
2021-03-13 01:26:07,437 - INFO - joeynmt.training - EPOCH 53
2021-03-13 01:26:46,440 - INFO - joeynmt.training - Epoch  53: total training loss 125.56
2021-03-13 01:26:46,441 - INFO - joeynmt.training - EPOCH 54
2021-03-13 01:27:25,431 - INFO - joeynmt.training - Epoch  54: total training loss 124.23
2021-03-13 01:27:25,431 - INFO - joeynmt.training - EPOCH 55
2021-03-13 01:28:04,880 - INFO - joeynmt.training - Epoch  55: total training loss 123.30
2021

2021-03-13 01:41:41,705 - INFO - joeynmt.training - Epoch  72: total training loss 103.23
2021-03-13 01:41:41,705 - INFO - joeynmt.training - EPOCH 73
2021-03-13 01:42:22,255 - INFO - joeynmt.training - Epoch  73: total training loss 101.90
2021-03-13 01:42:22,255 - INFO - joeynmt.training - EPOCH 74
2021-03-13 01:43:01,738 - INFO - joeynmt.training - Epoch  74: total training loss 100.77
2021-03-13 01:43:01,739 - INFO - joeynmt.training - EPOCH 75
2021-03-13 01:43:42,245 - INFO - joeynmt.training - Epoch  75: total training loss 99.88
2021-03-13 01:43:42,245 - INFO - joeynmt.training - EPOCH 76
2021-03-13 01:44:24,770 - INFO - joeynmt.training - Epoch  76: total training loss 98.67
2021-03-13 01:44:24,770 - INFO - joeynmt.training - EPOCH 77
2021-03-13 01:45:06,455 - INFO - joeynmt.training - Epoch  77: total training loss 97.87
2021-03-13 01:45:06,456 - INFO - joeynmt.training - EPOCH 78
2021-03-13 01:45:48,960 - INFO - joeynmt.training - Epoch  78: total training loss 97.21
2021-03-

...

2021-03-13 03:20:18,462 - INFO - joeynmt.training - Epoch 194: total training loss 45.81
2021-03-13 03:20:18,462 - INFO - joeynmt.training - EPOCH 195
2021-03-13 03:20:59,159 - INFO - joeynmt.training - Epoch 195: total training loss 45.59
2021-03-13 03:20:59,160 - INFO - joeynmt.training - EPOCH 196
2021-03-13 03:21:38,220 - INFO - joeynmt.training - Epoch 196: total training loss 45.54
2021-03-13 03:21:38,221 - INFO - joeynmt.training - EPOCH 197
2021-03-13 03:22:17,010 - INFO - joeynmt.training - Epoch 197: total training loss 45.20
2021-03-13 03:22:17,010 - INFO - joeynmt.training - EPOCH 198
2021-03-13 03:23:46,464 - INFO - joeynmt.training - Example #0
2021-03-13 03:23:46,465 - INFO - joeynmt.training - 	Source:     duerme mucho de día
2021-03-13 03:23:46,465 - INFO - joeynmt.training - 	Reference:  ayoy xëëm nyëkë ma'ay
2021-03-13 03:23:46,465 - INFO - joeynmt.training - 	Hypothesis: ayoy koots nyëkë ma'ay
2021-03-13 03:23:46,465 - INFO - joeynmt.training - Example #1
2021-03-13

2021-03-13 03:39:20,729 - INFO - joeynmt.training - Epoch 219: total training loss 41.74
2021-03-13 03:39:20,730 - INFO - joeynmt.training - EPOCH 220
2021-03-13 03:40:02,495 - INFO - joeynmt.training - Epoch 220: total training loss 41.34
2021-03-13 03:40:02,495 - INFO - joeynmt.training - EPOCH 221
2021-03-13 03:41:41,770 - INFO - joeynmt.training - Example #0
2021-03-13 03:41:41,770 - INFO - joeynmt.training - 	Source:     duerme mucho de día
2021-03-13 03:41:41,770 - INFO - joeynmt.training - 	Reference:  ayoy xëëm nyëkë ma'ay
2021-03-13 03:41:41,770 - INFO - joeynmt.training - 	Hypothesis: xëëm yë'ë mya'ay
2021-03-13 03:41:41,770 - INFO - joeynmt.training - Example #1
2021-03-13 03:41:41,770 - INFO - joeynmt.training - 	Source:     cuando los soldados hubieron crucificado a jesús, tomaron sus vestidos, e hicieron cuatro partes, una para cada soldado
2021-03-13 03:41:41,770 - INFO - joeynmt.training - 	Reference:  ku soldadodëjk ja jesus ajxy yꞌukmëjpaꞌabejty cruz këxp, mënitë jesu

2021-03-13 03:59:23,992 - INFO - joeynmt.training - Example #0
2021-03-13 03:59:23,992 - INFO - joeynmt.training - 	Source:     duerme mucho de día
2021-03-13 03:59:23,993 - INFO - joeynmt.training - 	Reference:  ayoy xëëm nyëkë ma'ay
2021-03-13 03:59:23,993 - INFO - joeynmt.training - 	Hypothesis: xëëm yë'ë mya'ay
2021-03-13 03:59:23,993 - INFO - joeynmt.training - Example #1
2021-03-13 03:59:23,993 - INFO - joeynmt.training - 	Source:     cuando los soldados hubieron crucificado a jesús, tomaron sus vestidos, e hicieron cuatro partes, una para cada soldado
2021-03-13 03:59:23,993 - INFO - joeynmt.training - 	Reference:  ku soldadodëjk ja jesus ajxy yꞌukmëjpaꞌabejty cruz këxp, mënitë jesus ja wyit ajxy wyidsëꞌkë a mëdaaxk këëtsy ajxy yajpëdsëëmy, këjtuꞌ këëtsy jatyë soldadodëjk ajxy jeꞌe yajnënyëwaꞌxëëyë
2021-03-13 03:59:23,993 - INFO - joeynmt.training - 	Hypothesis: ku ja soldadodëjk ajxy ënajty të yꞌuknëgywendëꞌadëgodyë, mënitë jesus ajxy yajnënyëmaꞌay ma ja tuumbëdëjk ajxyën, jëdu

2021-03-13 04:17:10,439 - INFO - joeynmt.training - 	Hypothesis: yaa jesus ja fariseo ajxy ënajty të tyëgëꞌëyëë ma ja fariseo ajxy��n, ooy ajxy ënajty yꞌabaady ¡mijts tsaꞌantyëjk!
2021-03-13 04:17:10,439 - INFO - joeynmt.training - Validation result (greedy) at epoch 268, step    11500: bleu:   8.53, loss: 91004.6094, ppl:  56.8151, duration: 61.0524s
2021-03-13 04:17:32,696 - INFO - joeynmt.training - Epoch 268: total training loss 35.45
2021-03-13 04:17:32,696 - INFO - joeynmt.training - EPOCH 269
2021-03-13 04:18:12,816 - INFO - joeynmt.training - Epoch 269: total training loss 35.56
2021-03-13 04:18:12,817 - INFO - joeynmt.training - EPOCH 270
2021-03-13 04:18:52,534 - INFO - joeynmt.training - Epoch 270: total training loss 35.34
2021-03-13 04:18:52,535 - INFO - joeynmt.training - EPOCH 271
2021-03-13 04:19:32,400 - INFO - joeynmt.training - Epoch 271: total training loss 35.31
2021-03-13 04:19:32,400 - INFO - joeynmt.training - EPOCH 272
2021-03-13 04:20:14,022 - INFO - joeynmt.t

2021-03-13 04:35:30,730 - INFO - joeynmt.training - Epoch 292: total training loss 33.67
2021-03-13 04:35:30,731 - INFO - joeynmt.training - EPOCH 293
2021-03-13 04:36:11,009 - INFO - joeynmt.training - Epoch 293: total training loss 33.72
2021-03-13 04:36:11,009 - INFO - joeynmt.training - EPOCH 294
2021-03-13 04:36:49,262 - INFO - joeynmt.training - Epoch 294: total training loss 33.39
2021-03-13 04:36:49,263 - INFO - joeynmt.training - EPOCH 295
2021-03-13 04:37:27,233 - INFO - joeynmt.training - Epoch 295: total training loss 33.47
2021-03-13 04:37:27,234 - INFO - joeynmt.training - EPOCH 296
2021-03-13 04:38:05,719 - INFO - joeynmt.training - Epoch 296: total training loss 33.41
2021-03-13 04:38:05,720 - INFO - joeynmt.training - EPOCH 297
2021-03-13 04:38:43,295 - INFO - joeynmt.training - Epoch 297: total training loss 33.43
2021-03-13 04:38:43,295 - INFO - joeynmt.training - EPOCH 298
2021-03-13 04:39:20,492 - INFO - joeynmt.training - Epoch 298: total training loss 33.39
2021-

In [17]:
! cd joeynmt; /ve/bin/python3 -m joeynmt translate models/transformer_colab4/config.yaml 

2021-03-16 22:43:38,907 - INFO - root - Hello! This is Joey-NMT (version 1.2).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/ve/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2021-03-16 22:43:39,426 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-03-16 22:43:39,757 - INFO - joeynmt.model - Enc-d

## Necesita el tokenizador sacrebleu  SIN RANDOM

### Con BPE 4000 100 epoch 70 longitud

In [34]:
#CONFIGURACIÓN DE WIXARIKA INAOE

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:                           # specify training details here
    #load_model: "models/sin-random1/.ckpt" # if given, load a pre-trained model from this checkpoint
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    adam_betas: [0.9, 0.999]        # beta parameters for Adam. These are the defaults. Typically these are different for Transformer models.
    learning_rate: 0.0003            # initial learning rate, default: 3.0e-4
    learning_rate_min: 0.00000001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.0                # l2 regularization, default: 0
    batch_size: 128                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token"). When you use more than 1 GPUs, the actual batch size per device will be: batch_size // n_gpu.
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 256             # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    normalization: "batch"          # loss normalization of a mini-batch, default: "batch" (by number of sequences in batch), other options: "tokens" (by number of tokens in batch), "none" (don't normalize, sum up loss)
    scheduling: "noam"              # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 5                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 100                     # train for this many epochs
    validation_freq: 500           # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    #early_stopping_metric: "eval_metric"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: "models/sin-random1" # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: True                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    fp16: False                      # whether to use 16-bit half-precision training (through NVIDIA apex) instead of 32-bit training.
    max_output_length: 70           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: [0, 1, 2]    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)
    
model:                              # specify your model architecture here
    initializer: "xavier"           # initializer for all trainable weights (xavier, zeros, normal, uniform)
    init_gain: 1.0                  # gain for Xavier initializer (default: 1.0)
    bias_initializer: "zeros"       # initializer for bias terms (xavier, zeros, normal, uniform)
    embed_initializer: "xavier"     # initializer for embeddings (xavier, zeros, normal, uniform)
    embed_init_gain: 1.0            # gain for Xavier initializer for embeddings (default: 1.0)
    tied_embeddings: False          # tie src and trg embeddings, only applicable if vocabularies are the same, default: False
    tied_softmax: True
    encoder:
        type: "transformer"          # encoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256       # size of embeddings (for Transformer set equal to hidden_size)
            scale: True             # scale the embeddings by sqrt of their size, default: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1                # apply dropout to the inputs to the RNN, default: 0.0
        freeze: False               # if True, encoder parameters are not updated during training (does not include embedding parameters)
    decoder:
        type: "transformer"         # decoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256
            scale: True
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1
        freeze: False               # if True, decoder parameters are not updated during training (does not include embedding parameters, but attention)

""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [35]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 03:32:29,943 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 03:32:30,037 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 03:32:30,332 - INFO - joeynmt.data - building v

2021-03-18 03:35:12,995 - INFO - joeynmt.training - Epoch   1: total training loss 9268.01
2021-03-18 03:35:12,995 - INFO - joeynmt.training - EPOCH 2
2021-03-18 03:37:02,529 - INFO - joeynmt.training - Epoch   2: total training loss 8390.70
2021-03-18 03:37:02,531 - INFO - joeynmt.training - EPOCH 3
2021-03-18 03:38:48,254 - INFO - joeynmt.training - Epoch   3: total training loss 7882.39
2021-03-18 03:38:48,254 - INFO - joeynmt.training - EPOCH 4
2021-03-18 03:40:38,586 - INFO - joeynmt.training - Epoch   4: total training loss 7757.50
2021-03-18 03:40:38,586 - INFO - joeynmt.training - EPOCH 5
2021-03-18 03:47:09,395 - INFO - joeynmt.training - Epoch   5: total training loss 7534.37
2021-03-18 03:47:09,396 - INFO - joeynmt.training - EPOCH 6
2021-03-18 03:49:26,447 - INFO - joeynmt.training - Epoch   6: total training loss 7099.86
2021-03-18 03:49:26,449 - INFO - joeynmt.training - EPOCH 7
2021-03-18 03:51:06,545 - INFO - joeynmt.training - Epoch   7: total training loss 6742.94
202

2021-03-18 04:23:34,608 - INFO - joeynmt.training - Epoch  24: total training loss 3498.92
2021-03-18 04:23:34,608 - INFO - joeynmt.training - EPOCH 25
2021-03-18 04:25:19,377 - INFO - joeynmt.training - Epoch  25: total training loss 3366.92
2021-03-18 04:25:19,377 - INFO - joeynmt.training - EPOCH 26
2021-03-18 04:27:00,915 - INFO - joeynmt.training - Epoch  26: total training loss 3161.68
2021-03-18 04:27:00,915 - INFO - joeynmt.training - EPOCH 27
2021-03-18 04:28:38,879 - INFO - joeynmt.training - Epoch  27: total training loss 3031.02
2021-03-18 04:28:38,879 - INFO - joeynmt.training - EPOCH 28
2021-03-18 04:30:23,499 - INFO - joeynmt.training - Epoch  28: total training loss 2850.74
2021-03-18 04:30:23,499 - INFO - joeynmt.training - EPOCH 29
2021-03-18 04:32:09,530 - INFO - joeynmt.training - Epoch  29: total training loss 2703.41
2021-03-18 04:32:09,530 - INFO - joeynmt.training - EPOCH 30
2021-03-18 04:33:49,645 - INFO - joeynmt.training - Epoch  30: total training loss 2573.

2021-03-18 05:16:54,164 - INFO - joeynmt.training - Epoch  48: total training loss 849.22
2021-03-18 05:16:54,164 - INFO - joeynmt.training - EPOCH 49
2021-03-18 05:18:34,549 - INFO - joeynmt.training - Epoch  49: total training loss 816.75
2021-03-18 05:18:34,549 - INFO - joeynmt.training - EPOCH 50
2021-03-18 05:20:16,608 - INFO - joeynmt.training - Epoch  50: total training loss 780.22
2021-03-18 05:20:16,608 - INFO - joeynmt.training - EPOCH 51
2021-03-18 05:21:53,177 - INFO - joeynmt.training - Epoch  51: total training loss 732.90
2021-03-18 05:21:53,177 - INFO - joeynmt.training - EPOCH 52
2021-03-18 05:23:32,921 - INFO - joeynmt.training - Epoch  52: total training loss 689.87
2021-03-18 05:23:32,922 - INFO - joeynmt.training - EPOCH 53
2021-03-18 05:25:18,451 - INFO - joeynmt.training - Epoch  53: total training loss 657.72
2021-03-18 05:25:18,453 - INFO - joeynmt.training - EPOCH 54
2021-03-18 05:27:00,240 - INFO - joeynmt.training - Epoch  54: total training loss 626.73
2021

2021-03-18 06:02:02,397 - INFO - joeynmt.training - Epoch  72: total training loss 340.99
2021-03-18 06:02:02,398 - INFO - joeynmt.training - EPOCH 73
2021-03-18 06:03:39,844 - INFO - joeynmt.training - Epoch  73: total training loss 336.88
2021-03-18 06:03:39,844 - INFO - joeynmt.training - EPOCH 74
2021-03-18 06:05:20,728 - INFO - joeynmt.training - Epoch  74: total training loss 327.91
2021-03-18 06:05:20,729 - INFO - joeynmt.training - EPOCH 75
2021-03-18 06:07:02,056 - INFO - joeynmt.training - Epoch  75: total training loss 324.67
2021-03-18 06:07:02,057 - INFO - joeynmt.training - EPOCH 76
2021-03-18 06:08:48,161 - INFO - joeynmt.training - Epoch  76: total training loss 302.75
2021-03-18 06:08:48,161 - INFO - joeynmt.training - EPOCH 77
2021-03-18 06:10:33,997 - INFO - joeynmt.training - Epoch  77: total training loss 312.08
2021-03-18 06:10:33,997 - INFO - joeynmt.training - EPOCH 78
2021-03-18 06:12:15,313 - INFO - joeynmt.training - Epoch  78: total training loss 299.65
2021

2021-03-18 06:53:29,990 - INFO - joeynmt.training - Epoch  96: total training loss 205.34
2021-03-18 06:53:29,990 - INFO - joeynmt.training - EPOCH 97
2021-03-18 06:55:17,621 - INFO - joeynmt.training - Epoch  97: total training loss 206.14
2021-03-18 06:55:17,621 - INFO - joeynmt.training - EPOCH 98
2021-03-18 06:57:02,814 - INFO - joeynmt.training - Epoch  98: total training loss 202.98
2021-03-18 06:57:02,815 - INFO - joeynmt.training - EPOCH 99
2021-03-18 06:58:48,713 - INFO - joeynmt.training - Epoch  99: total training loss 202.22
2021-03-18 06:58:48,714 - INFO - joeynmt.training - EPOCH 100
2021-03-18 07:00:33,529 - INFO - joeynmt.training - Epoch 100: total training loss 194.60
2021-03-18 07:00:33,529 - INFO - joeynmt.training - Training ended after 100 epochs.
2021-03-18 07:00:33,529 - INFO - joeynmt.training - Best validation result (greedy) at step     4000:   8.12 eval_metric.
2021-03-18 07:00:33,555 - INFO - joeynmt.prediction - Process device: cpu, n_gpu: 0, batch_size pe

### con bpe 4000 100 epoch 50 longitud

In [36]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:                           # specify training details here
    #load_model: "models/sin-random2/.ckpt" # if given, load a pre-trained model from this checkpoint
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    adam_betas: [0.9, 0.999]        # beta parameters for Adam. These are the defaults. Typically these are different for Transformer models.
    learning_rate: 0.0003            # initial learning rate, default: 3.0e-4
    learning_rate_min: 0.00000001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.0                # l2 regularization, default: 0
    batch_size: 128                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token"). When you use more than 1 GPUs, the actual batch size per device will be: batch_size // n_gpu.
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 256             # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    normalization: "batch"          # loss normalization of a mini-batch, default: "batch" (by number of sequences in batch), other options: "tokens" (by number of tokens in batch), "none" (don't normalize, sum up loss)
    scheduling: "noam"              # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 5                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 100                     # train for this many epochs
    validation_freq: 500           # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    #early_stopping_metric: "eval_metric"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: "models/sin-random2" # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: True                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    fp16: False                      # whether to use 16-bit half-precision training (through NVIDIA apex) instead of 32-bit training.
    max_output_length: 50           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: [0, 1, 2]    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)
    
model:                              # specify your model architecture here
    initializer: "xavier"           # initializer for all trainable weights (xavier, zeros, normal, uniform)
    init_gain: 1.0                  # gain for Xavier initializer (default: 1.0)
    bias_initializer: "zeros"       # initializer for bias terms (xavier, zeros, normal, uniform)
    embed_initializer: "xavier"     # initializer for embeddings (xavier, zeros, normal, uniform)
    embed_init_gain: 1.0            # gain for Xavier initializer for embeddings (default: 1.0)
    tied_embeddings: False          # tie src and trg embeddings, only applicable if vocabularies are the same, default: False
    tied_softmax: True
    encoder:
        type: "transformer"          # encoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256       # size of embeddings (for Transformer set equal to hidden_size)
            scale: True             # scale the embeddings by sqrt of their size, default: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1                # apply dropout to the inputs to the RNN, default: 0.0
        freeze: False               # if True, encoder parameters are not updated during training (does not include embedding parameters)
    decoder:
        type: "transformer"         # decoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256
            scale: True
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1
        freeze: False               # if True, decoder parameters are not updated during training (does not include embedding parameters, but attention)

""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [37]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 07:05:40,197 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 07:05:40,258 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 07:05:40,515 - INFO - joeynmt.data - building v

2021-03-18 07:07:13,449 - INFO - joeynmt.training - Epoch   1: total training loss 6924.35
2021-03-18 07:07:13,449 - INFO - joeynmt.training - EPOCH 2
2021-03-18 07:08:32,767 - INFO - joeynmt.training - Epoch   2: total training loss 6344.53
2021-03-18 07:08:32,768 - INFO - joeynmt.training - EPOCH 3
2021-03-18 07:09:47,884 - INFO - joeynmt.training - Epoch   3: total training loss 5868.01
2021-03-18 07:09:47,885 - INFO - joeynmt.training - EPOCH 4
2021-03-18 07:11:05,089 - INFO - joeynmt.training - Epoch   4: total training loss 5772.16
2021-03-18 07:11:05,090 - INFO - joeynmt.training - EPOCH 5
2021-03-18 07:12:20,537 - INFO - joeynmt.training - Epoch   5: total training loss 5676.04
2021-03-18 07:12:20,538 - INFO - joeynmt.training - EPOCH 6
2021-03-18 07:13:39,465 - INFO - joeynmt.training - Epoch   6: total training loss 5496.86
2021-03-18 07:13:39,466 - INFO - joeynmt.training - EPOCH 7
2021-03-18 07:14:58,224 - INFO - joeynmt.training - Epoch   7: total training loss 5216.05
202

2021-03-18 07:44:07,908 - INFO - joeynmt.training - Epoch  28: total training loss 1930.38
2021-03-18 07:44:07,908 - INFO - joeynmt.training - EPOCH 29
2021-03-18 07:45:29,210 - INFO - joeynmt.training - Epoch  29: total training loss 1832.14
2021-03-18 07:45:29,211 - INFO - joeynmt.training - EPOCH 30
2021-03-18 07:46:45,229 - INFO - joeynmt.training - Epoch  30: total training loss 1683.66
2021-03-18 07:46:45,230 - INFO - joeynmt.training - EPOCH 31
2021-03-18 07:48:03,425 - INFO - joeynmt.training - Epoch  31: total training loss 1583.56
2021-03-18 07:48:03,425 - INFO - joeynmt.training - EPOCH 32
2021-03-18 07:49:20,215 - INFO - joeynmt.training - Epoch  32: total training loss 1447.19
2021-03-18 07:49:20,215 - INFO - joeynmt.training - EPOCH 33
2021-03-18 07:50:35,013 - INFO - joeynmt.training - Epoch  33: total training loss 1325.93
2021-03-18 07:50:35,013 - INFO - joeynmt.training - EPOCH 34
2021-03-18 07:51:54,475 - INFO - joeynmt.training - Epoch  34: total training loss 1225.

2021-03-18 08:27:05,910 - INFO - joeynmt.training - Epoch  55: total training loss 284.94
2021-03-18 08:27:05,910 - INFO - joeynmt.training - EPOCH 56
2021-03-18 08:28:24,336 - INFO - joeynmt.training - Epoch  56: total training loss 270.41
2021-03-18 08:28:24,337 - INFO - joeynmt.training - EPOCH 57
2021-03-18 08:29:42,550 - INFO - joeynmt.training - Epoch  57: total training loss 256.92
2021-03-18 08:29:42,550 - INFO - joeynmt.training - EPOCH 58
2021-03-18 08:31:01,269 - INFO - joeynmt.training - Epoch  58: total training loss 251.44
2021-03-18 08:31:01,270 - INFO - joeynmt.training - EPOCH 59
2021-03-18 08:32:21,285 - INFO - joeynmt.training - Epoch  59: total training loss 241.09
2021-03-18 08:32:21,285 - INFO - joeynmt.training - EPOCH 60
2021-03-18 08:33:40,168 - INFO - joeynmt.training - Epoch  60: total training loss 229.70
2021-03-18 08:33:40,168 - INFO - joeynmt.training - EPOCH 61
2021-03-18 08:35:03,575 - INFO - joeynmt.training - Epoch  61: total training loss 217.41
2021

2021-03-18 09:06:12,089 - INFO - joeynmt.training - Epoch  82: total training loss 118.86
2021-03-18 09:06:12,089 - INFO - joeynmt.training - EPOCH 83
2021-03-18 09:07:34,879 - INFO - joeynmt.training - Epoch  83: total training loss 112.83
2021-03-18 09:07:34,880 - INFO - joeynmt.training - EPOCH 84
2021-03-18 09:08:55,582 - INFO - joeynmt.training - Epoch  84: total training loss 112.59
2021-03-18 09:08:55,582 - INFO - joeynmt.training - EPOCH 85
2021-03-18 09:10:10,591 - INFO - joeynmt.training - Epoch  85: total training loss 112.10
2021-03-18 09:10:10,591 - INFO - joeynmt.training - EPOCH 86
2021-03-18 09:11:37,300 - INFO - joeynmt.training - Epoch  86: total training loss 107.68
2021-03-18 09:11:37,301 - INFO - joeynmt.training - EPOCH 87
2021-03-18 09:12:53,699 - INFO - joeynmt.training - Epoch  87: total training loss 103.12
2021-03-18 09:12:53,699 - INFO - joeynmt.training - EPOCH 88
2021-03-18 09:14:07,407 - INFO - joeynmt.training - Epoch  88: total training loss 100.74
2021

In [38]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:                           # specify training details here
    #load_model: "models/sin-random3/.ckpt" # if given, load a pre-trained model from this checkpoint
    reset_best_ckpt: False          # if True, reset the tracking of the best checkpoint and scores. Use for domain adaptation or fine-tuning with new metrics or dev data.
    reset_scheduler: False          # if True, overwrite scheduler in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    reset_optimizer: False          # if True, overwrite optimizer in loaded checkpoint with parameters specified in this config. Use for domain adaptation or fine-tuning.
    random_seed: 42                 # set this seed to make training deterministic
    optimizer: "adam"               # choices: "sgd", "adam", "adadelta", "adagrad", "rmsprop", default is SGD
    adam_betas: [0.9, 0.999]        # beta parameters for Adam. These are the defaults. Typically these are different for Transformer models.
    learning_rate: 0.0003            # initial learning rate, default: 3.0e-4
    learning_rate_min: 0.00000001       # stop learning when learning rate is reduced below this threshold, default: 1.0e-8
    learning_rate_factor: 1        # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000     # warmup steps for Noam scheduler (used with Transformer)
    clip_grad_val: 1.0              # clip the gradients to this value when they exceed it, optional
    #clip_grad_norm: 1.0            # norm clipping instead of value clipping
    weight_decay: 0.0                # l2 regularization, default: 0
    batch_size: 128                  # mini-batch size as number of sentences (when batch_type is "sentence"; default) or total number of tokens (when batch_type is "token"). When you use more than 1 GPUs, the actual batch size per device will be: batch_size // n_gpu.
    batch_type: "sentence"          # create batches with sentences ("sentence", default) or tokens ("token")
    eval_batch_size: 128             # mini-batch size for evaluation (see batch_size above)
    eval_batch_type: "sentence"     # evaluation batch type ("sentence", default) or tokens ("token")
    batch_multiplier: 1             # increase the effective batch size with values >1 to batch_multiplier*batch_size without increasing memory consumption by making updates only every batch_multiplier batches
    normalization: "batch"          # loss normalization of a mini-batch, default: "batch" (by number of sequences in batch), other options: "tokens" (by number of tokens in batch), "none" (don't normalize, sum up loss)
    scheduling: "noam"              # learning rate scheduling, optional, if not specified stays constant, options: "plateau", "exponential", "decaying", "noam" (for Transformer), "warmupexponentialdecay"
    patience: 5                     # specific to plateau scheduler: wait for this many validations without improvement before decreasing the learning rate
    decrease_factor: 0.5            # specific to plateau & exponential scheduler: decrease the learning rate by this factor
    epochs: 100                     # train for this many epochs
    validation_freq: 500           # validate after this many updates (number of mini-batches), default: 1000
    logging_freq: 1000               # log the training progress after this many updates, default: 100
    eval_metric: "bleu"             # validation metric, default: "bleu", other options: "chrf", "token_accuracy", "sequence_accuracy"
    early_stopping_metric: "ppl"   # when a new high score on this metric is achieved, a checkpoint is written, when "eval_metric" (default) is maximized, when "loss" or "ppl" is minimized
    model_dir: "models/sin-random3" # directory where models and validation results are stored, required
    overwrite: True                 # overwrite existing model directory, default: False. Do not set to True unless for debugging!
    shuffle: True                   # shuffle the training data, default: True
    use_cuda: True                  # use CUDA for acceleration on GPU, required. Set to False when working on CPU.
    fp16: False                      # whether to use 16-bit half-precision training (through NVIDIA apex) instead of 32-bit training.
    max_output_length: 50           # maximum output length for decoding, default: None. If set to None, allow sentences of max 1.5*src length
    print_valid_sents: [0, 1, 2]    # print this many validation sentences during each validation run, default: [0, 1, 2]
    keep_last_ckpts: 3              # keep this many of the latest checkpoints, if -1: all of them, default: 5
    label_smoothing: 0.0            # label smoothing: reference tokens will have 1-label_smoothing probability instead of 1, rest of probability mass is uniformly distributed over the rest of the vocabulary, default: 0.0 (off)
    
model:                              # specify your model architecture here
    initializer: "xavier"           # initializer for all trainable weights (xavier, zeros, normal, uniform)
    init_gain: 1.0                  # gain for Xavier initializer (default: 1.0)
    bias_initializer: "zeros"       # initializer for bias terms (xavier, zeros, normal, uniform)
    embed_initializer: "xavier"     # initializer for embeddings (xavier, zeros, normal, uniform)
    embed_init_gain: 1.0            # gain for Xavier initializer for embeddings (default: 1.0)
    tied_embeddings: False          # tie src and trg embeddings, only applicable if vocabularies are the same, default: False
    tied_softmax: True
    encoder:
        type: "transformer"          # encoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256       # size of embeddings (for Transformer set equal to hidden_size)
            scale: True             # scale the embeddings by sqrt of their size, default: False
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1                # apply dropout to the inputs to the RNN, default: 0.0
        freeze: False               # if True, encoder parameters are not updated during training (does not include embedding parameters)
    decoder:
        type: "transformer"         # decoder type: "recurrent" for LSTM or GRU, or "transformer" for a Transformer
        num_layers: 6               # number of layers
        num_heads: 4                # number of transformer heads
        embeddings:
            embedding_dim: 256
            scale: True
            freeze: False           # if True, embeddings are not updated during training
        hidden_size: 256             # size of hidden layer; must be divisible by number of heads
        ff_size: 1024                # size of position-wise feed-forward layer
        dropout: 0.1
        freeze: False               # if True, decoder parameters are not updated during training (does not include embedding parameters, but attention)

""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [39]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 09:48:09,867 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 09:48:09,943 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 09:48:10,212 - INFO - joeynmt.data - building v

2021-03-18 09:49:35,697 - INFO - joeynmt.training - Epoch   1: total training loss 6954.15
2021-03-18 09:49:35,697 - INFO - joeynmt.training - EPOCH 2
2021-03-18 09:50:53,758 - INFO - joeynmt.training - Epoch   2: total training loss 6487.49
2021-03-18 09:50:53,760 - INFO - joeynmt.training - EPOCH 3
2021-03-18 09:52:07,812 - INFO - joeynmt.training - Epoch   3: total training loss 5979.10
2021-03-18 09:52:07,813 - INFO - joeynmt.training - EPOCH 4
2021-03-18 09:53:27,048 - INFO - joeynmt.training - Epoch   4: total training loss 5816.25
2021-03-18 09:53:27,048 - INFO - joeynmt.training - EPOCH 5
2021-03-18 09:54:43,129 - INFO - joeynmt.training - Epoch   5: total training loss 5719.32
2021-03-18 09:54:43,129 - INFO - joeynmt.training - EPOCH 6
2021-03-18 09:56:01,975 - INFO - joeynmt.training - Epoch   6: total training loss 5599.82
2021-03-18 09:56:01,975 - INFO - joeynmt.training - EPOCH 7
2021-03-18 09:57:23,273 - INFO - joeynmt.training - Epoch   7: total training loss 5435.39
202

2021-03-18 10:34:32,229 - INFO - joeynmt.training - Epoch  28: total training loss 2167.64
2021-03-18 10:34:32,230 - INFO - joeynmt.training - EPOCH 29
2021-03-18 10:36:23,536 - INFO - joeynmt.training - Epoch  29: total training loss 2085.79
2021-03-18 10:36:23,537 - INFO - joeynmt.training - EPOCH 30
2021-03-18 10:41:05,915 - INFO - joeynmt.training - Epoch  30: total training loss 1921.14
2021-03-18 10:41:05,915 - INFO - joeynmt.training - EPOCH 31
2021-03-18 10:42:40,395 - INFO - joeynmt.training - Epoch  31: total training loss 1773.56
2021-03-18 10:42:40,396 - INFO - joeynmt.training - EPOCH 32
2021-03-18 10:44:22,286 - INFO - joeynmt.training - Epoch  32: total training loss 1606.84
2021-03-18 10:44:22,286 - INFO - joeynmt.training - EPOCH 33
2021-03-18 10:46:15,608 - INFO - joeynmt.training - Epoch  33: total training loss 1488.97
2021-03-18 10:46:15,609 - INFO - joeynmt.training - EPOCH 34
2021-03-18 10:48:12,528 - INFO - joeynmt.training - Epoch  34: total training loss 1373.

2021-03-18 11:31:01,555 - INFO - joeynmt.training - Epoch  55: total training loss 267.04
2021-03-18 11:31:01,555 - INFO - joeynmt.training - EPOCH 56
2021-03-18 11:36:35,873 - INFO - joeynmt.training - Epoch  56: total training loss 244.58
2021-03-18 11:36:35,873 - INFO - joeynmt.training - EPOCH 57
2021-03-18 11:37:40,521 - INFO - joeynmt.training - Epoch  57: total training loss 228.48
2021-03-18 11:37:40,522 - INFO - joeynmt.training - EPOCH 58
2021-03-18 11:39:33,383 - INFO - joeynmt.training - Epoch  58: total training loss 222.52
2021-03-18 11:39:33,384 - INFO - joeynmt.training - EPOCH 59
2021-03-18 11:41:22,100 - INFO - joeynmt.training - Epoch  59: total training loss 208.59
2021-03-18 11:41:22,100 - INFO - joeynmt.training - EPOCH 60
2021-03-18 11:43:17,864 - INFO - joeynmt.training - Epoch  60: total training loss 197.49
2021-03-18 11:43:17,864 - INFO - joeynmt.training - EPOCH 61
2021-03-18 11:45:05,149 - INFO - joeynmt.training - Epoch  61: total training loss 194.67
2021

2021-03-18 12:31:17,221 - INFO - joeynmt.training - Epoch  82: total training loss 102.09
2021-03-18 12:31:17,222 - INFO - joeynmt.training - EPOCH 83
2021-03-18 12:33:22,568 - INFO - joeynmt.training - Epoch  83: total training loss 99.39
2021-03-18 12:33:22,568 - INFO - joeynmt.training - EPOCH 84
2021-03-18 12:35:20,134 - INFO - joeynmt.training - Epoch  84: total training loss 98.23
2021-03-18 12:35:20,134 - INFO - joeynmt.training - EPOCH 85
2021-03-18 12:37:20,085 - INFO - joeynmt.training - Epoch  85: total training loss 96.72
2021-03-18 12:37:20,086 - INFO - joeynmt.training - EPOCH 86
2021-03-18 12:39:21,064 - INFO - joeynmt.training - Epoch  86: total training loss 93.68
2021-03-18 12:39:21,064 - INFO - joeynmt.training - EPOCH 87
2021-03-18 12:41:26,390 - INFO - joeynmt.training - Epoch  87: total training loss 94.79
2021-03-18 12:41:26,390 - INFO - joeynmt.training - EPOCH 88
2021-03-18 12:43:20,297 - INFO - joeynmt.training - Epoch  88: total training loss 88.93
2021-03-18

### El bueno creo con 50 longitud como en el anterior

In [42]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/sin-random4/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/sin-random4"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [43]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 14:38:19,411 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 14:38:19,461 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 14:38:19,726 - INFO - joeynmt.data - building v

2021-03-18 14:38:59,781 - INFO - joeynmt.training - Epoch   1: total training loss 230.77
2021-03-18 14:38:59,782 - INFO - joeynmt.training - EPOCH 2
2021-03-18 14:39:35,572 - INFO - joeynmt.training - Epoch   2: total training loss 212.13
2021-03-18 14:39:35,573 - INFO - joeynmt.training - EPOCH 3
2021-03-18 14:40:11,017 - INFO - joeynmt.training - Epoch   3: total training loss 210.06
2021-03-18 14:40:11,018 - INFO - joeynmt.training - EPOCH 4
2021-03-18 14:40:46,973 - INFO - joeynmt.training - Epoch   4: total training loss 208.15
2021-03-18 14:40:46,973 - INFO - joeynmt.training - EPOCH 5
2021-03-18 14:41:22,623 - INFO - joeynmt.training - Epoch   5: total training loss 205.24
2021-03-18 14:41:22,623 - INFO - joeynmt.training - EPOCH 6
2021-03-18 14:41:59,738 - INFO - joeynmt.training - Epoch   6: total training loss 201.68
2021-03-18 14:41:59,738 - INFO - joeynmt.training - EPOCH 7
2021-03-18 14:42:36,864 - INFO - joeynmt.training - Epoch   7: total training loss 198.41
2021-03-18

2021-03-18 14:56:47,844 - INFO - joeynmt.training - Epoch  28: total training loss 147.20
2021-03-18 14:56:47,845 - INFO - joeynmt.training - EPOCH 29
2021-03-18 14:57:25,492 - INFO - joeynmt.training - Epoch  29: total training loss 145.67
2021-03-18 14:57:25,492 - INFO - joeynmt.training - EPOCH 30
2021-03-18 14:58:02,614 - INFO - joeynmt.training - Epoch  30: total training loss 143.58
2021-03-18 14:58:02,616 - INFO - joeynmt.training - EPOCH 31
2021-03-18 14:58:39,685 - INFO - joeynmt.training - Epoch  31: total training loss 142.70
2021-03-18 14:58:39,685 - INFO - joeynmt.training - EPOCH 32
2021-03-18 14:59:16,980 - INFO - joeynmt.training - Epoch  32: total training loss 140.67
2021-03-18 14:59:16,980 - INFO - joeynmt.training - EPOCH 33
2021-03-18 14:59:53,713 - INFO - joeynmt.training - Epoch  33: total training loss 138.18
2021-03-18 14:59:53,713 - INFO - joeynmt.training - EPOCH 34
2021-03-18 15:00:30,716 - INFO - joeynmt.training - Epoch  34: total training loss 136.65
2021

2021-03-18 15:14:59,411 - INFO - joeynmt.training - Epoch  55: total training loss 104.78
2021-03-18 15:14:59,411 - INFO - joeynmt.training - EPOCH 56
2021-03-18 15:15:36,853 - INFO - joeynmt.training - Epoch  56: total training loss 103.25
2021-03-18 15:15:36,853 - INFO - joeynmt.training - EPOCH 57
2021-03-18 15:16:13,180 - INFO - joeynmt.training - Epoch  57: total training loss 102.41
2021-03-18 15:16:13,180 - INFO - joeynmt.training - EPOCH 58
2021-03-18 15:16:50,680 - INFO - joeynmt.training - Epoch  58: total training loss 101.14
2021-03-18 15:16:50,680 - INFO - joeynmt.training - EPOCH 59
2021-03-18 15:17:27,745 - INFO - joeynmt.training - Epoch  59: total training loss 100.02
2021-03-18 15:17:27,746 - INFO - joeynmt.training - EPOCH 60
2021-03-18 15:18:05,640 - INFO - joeynmt.training - Epoch  60: total training loss 98.41
2021-03-18 15:18:05,640 - INFO - joeynmt.training - EPOCH 61
2021-03-18 15:18:43,011 - INFO - joeynmt.training - Epoch  61: total training loss 97.61
2021-0

2021-03-18 15:33:26,936 - INFO - joeynmt.training - Epoch  82: total training loss 78.05
2021-03-18 15:33:26,936 - INFO - joeynmt.training - EPOCH 83
2021-03-18 15:34:04,002 - INFO - joeynmt.training - Epoch  83: total training loss 76.80
2021-03-18 15:34:04,002 - INFO - joeynmt.training - EPOCH 84
2021-03-18 15:34:41,080 - INFO - joeynmt.training - Epoch  84: total training loss 76.21
2021-03-18 15:34:41,080 - INFO - joeynmt.training - EPOCH 85
2021-03-18 15:35:19,374 - INFO - joeynmt.training - Epoch  85: total training loss 75.36
2021-03-18 15:35:19,375 - INFO - joeynmt.training - EPOCH 86
2021-03-18 15:35:58,160 - INFO - joeynmt.training - Epoch  86: total training loss 74.29
2021-03-18 15:35:58,161 - INFO - joeynmt.training - EPOCH 87
2021-03-18 15:36:36,304 - INFO - joeynmt.training - Epoch  87: total training loss 73.74
2021-03-18 15:36:36,304 - INFO - joeynmt.training - EPOCH 88
2021-03-18 15:37:15,251 - INFO - joeynmt.training - Epoch  88: total training loss 72.89
2021-03-18 

### con longitud 70

In [44]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/sin-random5/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/sin-random5"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [45]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 15:49:34,398 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 15:49:34,461 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 15:49:34,746 - INFO - joeynmt.data - building v

2021-03-18 15:50:36,174 - INFO - joeynmt.training - Epoch   1: total training loss 260.78
2021-03-18 15:50:36,176 - INFO - joeynmt.training - EPOCH 2
2021-03-18 15:51:29,437 - INFO - joeynmt.training - Epoch   2: total training loss 241.41
2021-03-18 15:51:29,437 - INFO - joeynmt.training - EPOCH 3
2021-03-18 15:52:23,648 - INFO - joeynmt.training - Epoch   3: total training loss 239.42
2021-03-18 15:52:23,649 - INFO - joeynmt.training - EPOCH 4
2021-03-18 15:53:15,604 - INFO - joeynmt.training - Epoch   4: total training loss 236.73
2021-03-18 15:53:15,605 - INFO - joeynmt.training - EPOCH 5
2021-03-18 15:54:05,913 - INFO - joeynmt.training - Epoch   5: total training loss 233.03
2021-03-18 15:54:05,913 - INFO - joeynmt.training - EPOCH 6
2021-03-18 15:54:56,267 - INFO - joeynmt.training - Epoch   6: total training loss 229.05
2021-03-18 15:54:56,267 - INFO - joeynmt.training - EPOCH 7
2021-03-18 15:55:41,956 - INFO - joeynmt.training - Epoch   7: total training loss 224.22
2021-03-18

2021-03-18 16:10:54,534 - INFO - joeynmt.training - Epoch  24: total training loss 173.27
2021-03-18 16:10:54,534 - INFO - joeynmt.training - EPOCH 25
2021-03-18 16:11:44,258 - INFO - joeynmt.training - Epoch  25: total training loss 170.52
2021-03-18 16:11:44,259 - INFO - joeynmt.training - EPOCH 26
2021-03-18 16:12:31,540 - INFO - joeynmt.training - Epoch  26: total training loss 168.18
2021-03-18 16:12:31,542 - INFO - joeynmt.training - EPOCH 27
2021-03-18 16:13:19,796 - INFO - joeynmt.training - Epoch  27: total training loss 166.77
2021-03-18 16:13:19,796 - INFO - joeynmt.training - EPOCH 28
2021-03-18 16:14:07,476 - INFO - joeynmt.training - Epoch  28: total training loss 163.76
2021-03-18 16:14:07,477 - INFO - joeynmt.training - EPOCH 29
2021-03-18 16:14:56,321 - INFO - joeynmt.training - Epoch  29: total training loss 160.90
2021-03-18 16:14:56,321 - INFO - joeynmt.training - EPOCH 30
2021-03-18 16:15:44,240 - INFO - joeynmt.training - Epoch  30: total training loss 159.26
2021

2021-03-18 16:32:45,724 - INFO - joeynmt.training - Epoch  48: total training loss 126.95
2021-03-18 16:32:45,725 - INFO - joeynmt.training - EPOCH 49
2021-03-18 16:33:37,312 - INFO - joeynmt.training - Epoch  49: total training loss 125.80
2021-03-18 16:33:37,312 - INFO - joeynmt.training - EPOCH 50
2021-03-18 16:34:28,774 - INFO - joeynmt.training - Epoch  50: total training loss 126.47
2021-03-18 16:34:28,774 - INFO - joeynmt.training - EPOCH 51
2021-03-18 16:35:20,958 - INFO - joeynmt.training - Epoch  51: total training loss 123.70
2021-03-18 16:35:20,959 - INFO - joeynmt.training - EPOCH 52
2021-03-18 16:36:12,765 - INFO - joeynmt.training - Epoch  52: total training loss 121.54
2021-03-18 16:36:12,765 - INFO - joeynmt.training - EPOCH 53
2021-03-18 16:37:04,086 - INFO - joeynmt.training - Epoch  53: total training loss 120.22
2021-03-18 16:37:04,086 - INFO - joeynmt.training - EPOCH 54
2021-03-18 16:37:55,220 - INFO - joeynmt.training - Epoch  54: total training loss 118.84
2021

2021-03-18 16:55:07,770 - INFO - joeynmt.training - Epoch  72: total training loss 99.98
2021-03-18 16:55:07,770 - INFO - joeynmt.training - EPOCH 73
2021-03-18 16:55:58,568 - INFO - joeynmt.training - Epoch  73: total training loss 98.51
2021-03-18 16:55:58,568 - INFO - joeynmt.training - EPOCH 74
2021-03-18 16:56:48,874 - INFO - joeynmt.training - Epoch  74: total training loss 97.41
2021-03-18 16:56:48,875 - INFO - joeynmt.training - EPOCH 75
2021-03-18 16:57:40,195 - INFO - joeynmt.training - Epoch  75: total training loss 96.65
2021-03-18 16:57:40,196 - INFO - joeynmt.training - EPOCH 76
2021-03-18 16:58:31,050 - INFO - joeynmt.training - Epoch  76: total training loss 95.76
2021-03-18 16:58:31,050 - INFO - joeynmt.training - EPOCH 77
2021-03-18 16:59:22,345 - INFO - joeynmt.training - Epoch  77: total training loss 94.94
2021-03-18 16:59:22,345 - INFO - joeynmt.training - EPOCH 78
2021-03-18 17:00:13,244 - INFO - joeynmt.training - Epoch  78: total training loss 93.67
2021-03-18 

2021-03-18 17:18:06,280 - INFO - joeynmt.training - Epoch  96: total training loss 80.55
2021-03-18 17:18:06,280 - INFO - joeynmt.training - EPOCH 97
2021-03-18 17:18:57,246 - INFO - joeynmt.training - Epoch  97: total training loss 79.82
2021-03-18 17:18:57,246 - INFO - joeynmt.training - EPOCH 98
2021-03-18 17:19:48,081 - INFO - joeynmt.training - Epoch  98: total training loss 79.46
2021-03-18 17:19:48,081 - INFO - joeynmt.training - EPOCH 99
2021-03-18 17:20:39,897 - INFO - joeynmt.training - Epoch  99: total training loss 78.70
2021-03-18 17:20:39,899 - INFO - joeynmt.training - EPOCH 100
2021-03-18 17:21:32,213 - INFO - joeynmt.training - Epoch 100: total training loss 78.12
2021-03-18 17:21:32,214 - INFO - joeynmt.training - Training ended after 100 epochs.
2021-03-18 17:21:32,214 - INFO - joeynmt.training - Best validation result (greedy) at step     4000:  36.74 ppl.
2021-03-18 17:21:32,236 - INFO - joeynmt.prediction - Process device: cpu, n_gpu: 0, batch_size per device: 256

### Con longitud 50 pero con batch 64 eval bacth 128  embeding dim 64 ff_size 128 como la del wixarika

In [48]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/sin-random6/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/sin-random6"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [49]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 17:41:53,150 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 17:41:53,208 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 17:41:53,470 - INFO - joeynmt.data - building v

2021-03-18 17:42:10,577 - INFO - joeynmt.training - Epoch   1: total training loss 255.10
2021-03-18 17:42:10,577 - INFO - joeynmt.training - EPOCH 2
2021-03-18 17:42:27,941 - INFO - joeynmt.training - Epoch   2: total training loss 237.38
2021-03-18 17:42:27,942 - INFO - joeynmt.training - EPOCH 3
2021-03-18 17:42:44,824 - INFO - joeynmt.training - Epoch   3: total training loss 225.17
2021-03-18 17:42:44,825 - INFO - joeynmt.training - EPOCH 4
2021-03-18 17:43:01,916 - INFO - joeynmt.training - Epoch   4: total training loss 218.08
2021-03-18 17:43:01,917 - INFO - joeynmt.training - EPOCH 5
2021-03-18 17:43:18,377 - INFO - joeynmt.training - Epoch   5: total training loss 214.38
2021-03-18 17:43:18,377 - INFO - joeynmt.training - EPOCH 6
2021-03-18 17:43:34,696 - INFO - joeynmt.training - Epoch   6: total training loss 212.73
2021-03-18 17:43:34,696 - INFO - joeynmt.training - EPOCH 7
2021-03-18 17:43:50,992 - INFO - joeynmt.training - Epoch   7: total training loss 211.88
2021-03-18

2021-03-18 17:49:42,131 - INFO - joeynmt.training - Epoch  28: total training loss 184.95
2021-03-18 17:49:42,131 - INFO - joeynmt.training - EPOCH 29
2021-03-18 17:49:57,832 - INFO - joeynmt.training - Epoch  29: total training loss 183.63
2021-03-18 17:49:57,832 - INFO - joeynmt.training - EPOCH 30
2021-03-18 17:50:14,039 - INFO - joeynmt.training - Epoch  30: total training loss 182.54
2021-03-18 17:50:14,039 - INFO - joeynmt.training - EPOCH 31
2021-03-18 17:50:29,719 - INFO - joeynmt.training - Epoch  31: total training loss 181.45
2021-03-18 17:50:29,719 - INFO - joeynmt.training - EPOCH 32
2021-03-18 17:50:45,271 - INFO - joeynmt.training - Epoch  32: total training loss 180.51
2021-03-18 17:50:45,271 - INFO - joeynmt.training - EPOCH 33
2021-03-18 17:51:00,977 - INFO - joeynmt.training - Epoch  33: total training loss 179.54
2021-03-18 17:51:00,977 - INFO - joeynmt.training - EPOCH 34
2021-03-18 17:51:16,255 - INFO - joeynmt.training - Epoch  34: total training loss 178.70
2021

2021-03-18 17:57:07,586 - INFO - joeynmt.training - Epoch  55: total training loss 165.58
2021-03-18 17:57:07,587 - INFO - joeynmt.training - EPOCH 56
2021-03-18 17:57:23,133 - INFO - joeynmt.training - Epoch  56: total training loss 164.98
2021-03-18 17:57:23,133 - INFO - joeynmt.training - EPOCH 57
2021-03-18 17:57:39,483 - INFO - joeynmt.training - Epoch  57: total training loss 164.43
2021-03-18 17:57:39,483 - INFO - joeynmt.training - EPOCH 58
2021-03-18 17:57:55,613 - INFO - joeynmt.training - Epoch  58: total training loss 163.93
2021-03-18 17:57:55,614 - INFO - joeynmt.training - EPOCH 59
2021-03-18 17:58:11,538 - INFO - joeynmt.training - Epoch  59: total training loss 163.58
2021-03-18 17:58:11,539 - INFO - joeynmt.training - EPOCH 60
2021-03-18 17:58:27,393 - INFO - joeynmt.training - Epoch  60: total training loss 163.05
2021-03-18 17:58:27,394 - INFO - joeynmt.training - EPOCH 61
2021-03-18 17:58:43,552 - INFO - joeynmt.training - Epoch  61: total training loss 162.48
2021

2021-03-18 18:04:38,850 - INFO - joeynmt.training - Epoch  82: total training loss 153.18
2021-03-18 18:04:38,850 - INFO - joeynmt.training - EPOCH 83
2021-03-18 18:04:54,568 - INFO - joeynmt.training - Epoch  83: total training loss 152.63
2021-03-18 18:04:54,569 - INFO - joeynmt.training - EPOCH 84
2021-03-18 18:05:10,174 - INFO - joeynmt.training - Epoch  84: total training loss 152.37
2021-03-18 18:05:10,174 - INFO - joeynmt.training - EPOCH 85
2021-03-18 18:05:25,864 - INFO - joeynmt.training - Epoch  85: total training loss 151.82
2021-03-18 18:05:25,864 - INFO - joeynmt.training - EPOCH 86
2021-03-18 18:05:41,812 - INFO - joeynmt.training - Epoch  86: total training loss 151.61
2021-03-18 18:05:41,813 - INFO - joeynmt.training - EPOCH 87
2021-03-18 18:05:57,652 - INFO - joeynmt.training - Epoch  87: total training loss 151.01
2021-03-18 18:05:57,652 - INFO - joeynmt.training - EPOCH 88
2021-03-18 18:06:13,488 - INFO - joeynmt.training - Epoch  88: total training loss 150.89
2021

# Con bpe 2000

In [58]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/sin-random7/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/sin-random7"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [59]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 18:56:32,736 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 18:56:32,787 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 18:56:33,145 - INFO - joeynmt.data - building v

2021-03-18 18:57:32,223 - INFO - joeynmt.training - Epoch   1: total training loss 197.90
2021-03-18 18:57:32,224 - INFO - joeynmt.training - EPOCH 2
2021-03-18 18:58:04,059 - INFO - joeynmt.training - Epoch   2: total training loss 183.52
2021-03-18 18:58:04,060 - INFO - joeynmt.training - EPOCH 3
2021-03-18 18:58:35,699 - INFO - joeynmt.training - Epoch   3: total training loss 182.00
2021-03-18 18:58:35,699 - INFO - joeynmt.training - EPOCH 4
2021-03-18 18:59:07,246 - INFO - joeynmt.training - Epoch   4: total training loss 180.18
2021-03-18 18:59:07,246 - INFO - joeynmt.training - EPOCH 5
2021-03-18 18:59:38,257 - INFO - joeynmt.training - Epoch   5: total training loss 177.78
2021-03-18 18:59:38,257 - INFO - joeynmt.training - EPOCH 6
2021-03-18 19:00:09,682 - INFO - joeynmt.training - Epoch   6: total training loss 175.60
2021-03-18 19:00:09,682 - INFO - joeynmt.training - EPOCH 7
2021-03-18 19:00:41,500 - INFO - joeynmt.training - Epoch   7: total training loss 172.84
2021-03-18

2021-03-18 19:16:01,813 - INFO - joeynmt.training - Epoch  30: total training loss 113.48
2021-03-18 19:16:01,813 - INFO - joeynmt.training - EPOCH 31
2021-03-18 19:16:31,331 - INFO - joeynmt.training - Epoch  31: total training loss 111.78
2021-03-18 19:16:31,332 - INFO - joeynmt.training - EPOCH 32
2021-03-18 19:17:02,537 - INFO - joeynmt.training - Epoch  32: total training loss 110.37
2021-03-18 19:17:02,537 - INFO - joeynmt.training - EPOCH 33
2021-03-18 19:17:33,295 - INFO - joeynmt.training - Epoch  33: total training loss 109.24
2021-03-18 19:17:33,295 - INFO - joeynmt.training - EPOCH 34
2021-03-18 19:18:02,894 - INFO - joeynmt.training - Epoch  34: total training loss 107.39
2021-03-18 19:18:02,895 - INFO - joeynmt.training - EPOCH 35
2021-03-18 19:18:33,042 - INFO - joeynmt.training - Epoch  35: total training loss 106.14
2021-03-18 19:18:33,043 - INFO - joeynmt.training - EPOCH 36
2021-03-18 19:19:02,533 - INFO - joeynmt.training - Epoch  36: total training loss 104.84
2021

2021-03-18 19:32:01,325 - INFO - joeynmt.training - Epoch  59: total training loss 82.42
2021-03-18 19:32:01,325 - INFO - joeynmt.training - EPOCH 60
2021-03-18 19:32:31,958 - INFO - joeynmt.training - Epoch  60: total training loss 81.46
2021-03-18 19:32:31,958 - INFO - joeynmt.training - EPOCH 61
2021-03-18 19:33:03,196 - INFO - joeynmt.training - Epoch  61: total training loss 80.55
2021-03-18 19:33:03,196 - INFO - joeynmt.training - EPOCH 62
2021-03-18 19:33:33,967 - INFO - joeynmt.training - Epoch  62: total training loss 80.69
2021-03-18 19:33:33,967 - INFO - joeynmt.training - EPOCH 63
2021-03-18 19:34:04,707 - INFO - joeynmt.training - Epoch  63: total training loss 79.98
2021-03-18 19:34:04,708 - INFO - joeynmt.training - EPOCH 64
2021-03-18 19:34:35,303 - INFO - joeynmt.training - Epoch  64: total training loss 78.89
2021-03-18 19:34:35,303 - INFO - joeynmt.training - EPOCH 65
2021-03-18 19:35:07,739 - INFO - joeynmt.training - Epoch  65: total training loss 77.87
2021-03-18 

2021-03-18 19:49:26,726 - INFO - joeynmt.training - Epoch  89: total training loss 63.71
2021-03-18 19:49:26,727 - INFO - joeynmt.training - EPOCH 90
2021-03-18 19:49:57,484 - INFO - joeynmt.training - Epoch  90: total training loss 63.04
2021-03-18 19:49:57,484 - INFO - joeynmt.training - EPOCH 91
2021-03-18 19:50:28,276 - INFO - joeynmt.training - Epoch  91: total training loss 62.56
2021-03-18 19:50:28,276 - INFO - joeynmt.training - EPOCH 92
2021-03-18 19:50:59,530 - INFO - joeynmt.training - Epoch  92: total training loss 62.35
2021-03-18 19:50:59,530 - INFO - joeynmt.training - EPOCH 93
2021-03-18 19:51:30,780 - INFO - joeynmt.training - Epoch  93: total training loss 61.83
2021-03-18 19:51:30,781 - INFO - joeynmt.training - EPOCH 94
2021-03-18 19:52:01,582 - INFO - joeynmt.training - Epoch  94: total training loss 61.19
2021-03-18 19:52:01,582 - INFO - joeynmt.training - EPOCH 95
2021-03-18 19:52:33,594 - INFO - joeynmt.training - Epoch  95: total training loss 60.56
2021-03-18 

In [60]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/sin-random8/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/sin-random8"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [61]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 19:58:31,644 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 19:58:31,760 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 19:58:32,050 - INFO - joeynmt.data - building v

2021-03-18 19:58:43,941 - INFO - joeynmt.training - Epoch   1: total training loss 215.85
2021-03-18 19:58:43,942 - INFO - joeynmt.training - EPOCH 2
2021-03-18 19:58:53,777 - INFO - joeynmt.training - Epoch   2: total training loss 201.04
2021-03-18 19:58:53,777 - INFO - joeynmt.training - EPOCH 3
2021-03-18 19:59:03,993 - INFO - joeynmt.training - Epoch   3: total training loss 191.42
2021-03-18 19:59:03,993 - INFO - joeynmt.training - EPOCH 4
2021-03-18 19:59:14,128 - INFO - joeynmt.training - Epoch   4: total training loss 186.70
2021-03-18 19:59:14,129 - INFO - joeynmt.training - EPOCH 5
2021-03-18 19:59:23,965 - INFO - joeynmt.training - Epoch   5: total training loss 184.75
2021-03-18 19:59:23,966 - INFO - joeynmt.training - EPOCH 6
2021-03-18 19:59:33,760 - INFO - joeynmt.training - Epoch   6: total training loss 183.96
2021-03-18 19:59:33,761 - INFO - joeynmt.training - EPOCH 7
2021-03-18 19:59:44,434 - INFO - joeynmt.training - Epoch   7: total training loss 183.50
2021-03-18

2021-03-18 20:04:21,152 - INFO - joeynmt.training - Epoch  30: total training loss 160.92
2021-03-18 20:04:21,153 - INFO - joeynmt.training - EPOCH 31
2021-03-18 20:04:31,515 - INFO - joeynmt.training - Epoch  31: total training loss 160.10
2021-03-18 20:04:31,515 - INFO - joeynmt.training - EPOCH 32
2021-03-18 20:04:41,473 - INFO - joeynmt.training - Epoch  32: total training loss 159.24
2021-03-18 20:04:41,473 - INFO - joeynmt.training - EPOCH 33
2021-03-18 20:04:51,726 - INFO - joeynmt.training - Epoch  33: total training loss 158.30
2021-03-18 20:04:51,727 - INFO - joeynmt.training - EPOCH 34
2021-03-18 20:05:01,883 - INFO - joeynmt.training - Epoch  34: total training loss 157.56
2021-03-18 20:05:01,883 - INFO - joeynmt.training - EPOCH 35
2021-03-18 20:05:11,746 - INFO - joeynmt.training - Epoch  35: total training loss 156.67
2021-03-18 20:05:11,747 - INFO - joeynmt.training - EPOCH 36
2021-03-18 20:05:22,109 - INFO - joeynmt.training - Epoch  36: total training loss 155.93
2021

2021-03-18 20:09:26,296 - INFO - joeynmt.training - 	Reference:  ja ajxy mëj ane'embë'ajpë ma jaty ja estados ja ane'emt kujaay kugeetsypë ajxy nyë'ëgë kudyunaampy, ja leyes mët ja ajxy të yajtëdyëgë'eyëbë, oy ja estados y'amdso ane'emt kujaay kugeetsypë o ja lyeyes ajxy ënajty tëgatsy jyanëjts'ane'emë'ëy
2021-03-18 20:09:26,296 - INFO - joeynmt.training - 	Hypothesis: ja ley ajxy jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un jëdu'un
2021-03-18 20:09:26,296 - INFO - joeynmt.training - Example #3
2021-03-18 20:09:26,296 - INFO - joeynmt.training - 	Source:     queda prohibido en todo centro de trabajo, el establecimiento de expendios de bebidas embriagantes y de casas de

2021-03-18 20:12:52,230 - INFO - joeynmt.training - Epoch  79: total training loss 135.79
2021-03-18 20:12:52,230 - INFO - joeynmt.training - EPOCH 80
2021-03-18 20:13:01,767 - INFO - joeynmt.training - Epoch  80: total training loss 135.34
2021-03-18 20:13:01,767 - INFO - joeynmt.training - EPOCH 81
2021-03-18 20:13:11,916 - INFO - joeynmt.training - Epoch  81: total training loss 135.14
2021-03-18 20:13:11,917 - INFO - joeynmt.training - EPOCH 82
2021-03-18 20:13:21,796 - INFO - joeynmt.training - Epoch  82: total training loss 134.66
2021-03-18 20:13:21,797 - INFO - joeynmt.training - EPOCH 83
2021-03-18 20:13:31,614 - INFO - joeynmt.training - Epoch  83: total training loss 134.30
2021-03-18 20:13:31,614 - INFO - joeynmt.training - EPOCH 84
2021-03-18 20:13:41,756 - INFO - joeynmt.training - Epoch  84: total training loss 134.04
2021-03-18 20:13:41,756 - INFO - joeynmt.training - EPOCH 85
2021-03-18 20:13:51,358 - INFO - joeynmt.training - Epoch  85: total training loss 133.72
2021

# CON RANDOM

### Con BPE 2000 len 50 pocas capas

In [70]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/con-random1/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/con-random1"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [71]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 20:29:59,435 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 20:29:59,500 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 20:29:59,765 - INFO - joeynmt.data - building v

2021-03-18 20:30:10,624 - INFO - joeynmt.training - Epoch   1: total training loss 220.95
2021-03-18 20:30:10,625 - INFO - joeynmt.training - EPOCH 2
2021-03-18 20:30:20,192 - INFO - joeynmt.training - Epoch   2: total training loss 205.33
2021-03-18 20:30:20,193 - INFO - joeynmt.training - EPOCH 3
2021-03-18 20:30:29,604 - INFO - joeynmt.training - Epoch   3: total training loss 195.38
2021-03-18 20:30:29,604 - INFO - joeynmt.training - EPOCH 4
2021-03-18 20:30:39,213 - INFO - joeynmt.training - Epoch   4: total training loss 190.76
2021-03-18 20:30:39,215 - INFO - joeynmt.training - EPOCH 5
2021-03-18 20:30:48,795 - INFO - joeynmt.training - Epoch   5: total training loss 188.97
2021-03-18 20:30:48,795 - INFO - joeynmt.training - EPOCH 6
2021-03-18 20:30:58,844 - INFO - joeynmt.training - Epoch   6: total training loss 188.45
2021-03-18 20:30:58,845 - INFO - joeynmt.training - EPOCH 7
2021-03-18 20:31:09,091 - INFO - joeynmt.training - Epoch   7: total training loss 188.06
2021-03-18

2021-03-18 20:35:05,009 - INFO - joeynmt.training - Epoch  29: total training loss 166.33
2021-03-18 20:35:05,009 - INFO - joeynmt.training - EPOCH 30
2021-03-18 20:35:14,027 - INFO - joeynmt.training - Epoch  30: total training loss 165.26
2021-03-18 20:35:14,027 - INFO - joeynmt.training - EPOCH 31
2021-03-18 20:35:23,326 - INFO - joeynmt.training - Epoch  31: total training loss 164.32
2021-03-18 20:35:23,326 - INFO - joeynmt.training - EPOCH 32
2021-03-18 20:35:32,494 - INFO - joeynmt.training - Epoch  32: total training loss 163.46
2021-03-18 20:35:32,494 - INFO - joeynmt.training - EPOCH 33
2021-03-18 20:35:41,934 - INFO - joeynmt.training - Epoch  33: total training loss 162.73
2021-03-18 20:35:41,934 - INFO - joeynmt.training - EPOCH 34
2021-03-18 20:35:51,587 - INFO - joeynmt.training - Epoch  34: total training loss 161.81
2021-03-18 20:35:51,587 - INFO - joeynmt.training - EPOCH 35
2021-03-18 20:36:00,869 - INFO - joeynmt.training - Epoch  35: total training loss 161.00
2021

2021-03-18 20:39:52,537 - INFO - joeynmt.training - 	Hypothesis: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a pën jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëdu��un jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun
2021-03-18 20:39:52,537 - INFO - joeynmt.training - Validation result (greedy) at epoch  58, step     2000: bleu:   0.37, loss: 121846.7891, ppl:  95.7760, duration: 12.9253s
2021-03-18 20:40:00,179 - INFO - joeynmt.training - Epoch  58: total training loss 148.75
2021-03-18 20:40:00,179 - INFO - joeynmt.training - EPOCH 59
2021-03-18 20:40:09,374 - INFO - joeynmt.training - Epoch  59: total training loss 148.28
2021-03-18 20:40:09,375 - INFO - joeynmt.training - EPOCH 60
2021-03-18 20:40:18,808 - INFO - joeynmt.training - Epoch  60: total training loss 147.71
2021-03-18 20:40:18,808 - INFO - joeynmt.training - EPOCH 61
2021-03-18 20:40:28,055 - INFO - joeynmt.training - Epoch  61: total training loss 147.51
2021-03-18 20:40:28,055 - INFO - joe

2021-03-18 20:44:45,285 - INFO - joeynmt.training - 	Hypothesis: pero pero pero jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun jëduꞌun të të të të të të të të të të të të të të të të të të të të të të të të të të të të nꞌiꞌꞌꞌꞌꞌ@@
2021-03-18 20:44:45,285 - INFO - joeynmt.training - Validation result (greedy) at epoch  86, step     3000: bleu:   1.05, loss: 114783.0938, ppl:  73.5190, duration: 13.6321s
2021-03-18 20:44:47,693 - INFO - joeynmt.training - Epoch  86: total training loss 137.39
2021-03-18 20:44:47,693 - INFO - joeynmt.training - EPOCH 87
2021-03-18 20:44:56,905 - INFO - joeynmt.training - Epoch  87: total training loss 137.17
2021-03-18 20:44:56,905 - INFO - joeynmt.training - EPOCH 88
2021-03-18 20:45:06,189 - INFO - joeynmt.training - Epoch  88: total training loss 136.63
2021-03-18 20:45:06,189 - INFO - joeynmt.training - EPOCH 89
2021-03-18 20:45:15,425 - INFO - joeynmt.training - Epoch  89: total training loss 136.35
2021-03-18 

## con bpe 2000 con muchas capas

In [72]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/con-random2/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/con-random2"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [73]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 20:51:53,124 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 20:51:53,178 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 20:51:53,441 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 20:52:25,824 - INFO - joeynmt.training - Epoch   1: total training loss 201.72
2021-03-18 20:52:25,825 - INFO - joeynmt.training - EPOCH 2
2021-03-18 20:52:55,193 - INFO - joeynmt.training - Epoch   2: total training loss 188.89
2021-03-18 20:52:55,194 - INFO - joeynmt.training - EPOCH 3
2021-03-18 20:53:24,604 - INFO - joeynmt.training - Epoch   3: total training loss 187.46
2021-03-18 20:53:24,604 - INFO - joeynmt.training - EPOCH 4
2021-03-18 20:53:53,893 - INFO - joeynmt.training - Epoch   4: total training loss 184.90
2021-03-18 20:53:53,893 - INFO - joeynmt.training - EP

2021-03-18 21:08:58,827 - INFO - joeynmt.training - Epoch  29: total training loss 121.19
2021-03-18 21:08:58,830 - INFO - joeynmt.training - EPOCH 30
2021-03-18 21:09:38,971 - INFO - joeynmt.training - Epoch  30: total training loss 119.70
2021-03-18 21:09:38,972 - INFO - joeynmt.training - EPOCH 31
2021-03-18 21:10:18,277 - INFO - joeynmt.training - Epoch  31: total training loss 118.61
2021-03-18 21:10:18,277 - INFO - joeynmt.training - EPOCH 32
2021-03-18 21:10:53,409 - INFO - joeynmt.training - Epoch  32: total training loss 116.30
2021-03-18 21:10:53,409 - INFO - joeynmt.training - EPOCH 33
2021-03-18 21:11:30,338 - INFO - joeynmt.training - Epoch  33: total training loss 114.67
2021-03-18 21:11:30,340 - INFO - joeynmt.training - EPOCH 34
2021-03-18 21:12:11,283 - INFO - joeynmt.training - Epoch  34: total training loss 112.54
2021-03-18 21:12:11,283 - INFO - joeynmt.training - EPOCH 35
2021-03-18 21:12:50,521 - INFO - joeynmt.training - Epoch  35: total training loss 110.96
2021

2021-03-18 21:26:48,520 - INFO - joeynmt.training - Epoch  58: total training loss 85.73
2021-03-18 21:26:48,520 - INFO - joeynmt.training - EPOCH 59
2021-03-18 21:27:17,099 - INFO - joeynmt.training - Epoch  59: total training loss 84.77
2021-03-18 21:27:17,100 - INFO - joeynmt.training - EPOCH 60
2021-03-18 21:27:45,420 - INFO - joeynmt.training - Epoch  60: total training loss 83.95
2021-03-18 21:27:45,421 - INFO - joeynmt.training - EPOCH 61
2021-03-18 21:28:14,678 - INFO - joeynmt.training - Epoch  61: total training loss 83.12
2021-03-18 21:28:14,679 - INFO - joeynmt.training - EPOCH 62
2021-03-18 21:28:43,917 - INFO - joeynmt.training - Epoch  62: total training loss 82.30
2021-03-18 21:28:43,918 - INFO - joeynmt.training - EPOCH 63
2021-03-18 21:29:13,888 - INFO - joeynmt.training - Epoch  63: total training loss 81.55
2021-03-18 21:29:13,888 - INFO - joeynmt.training - EPOCH 64
2021-03-18 21:29:43,998 - INFO - joeynmt.training - Epoch  64: total training loss 80.81
2021-03-18 

2021-03-18 21:44:47,723 - INFO - joeynmt.training - Epoch  86: total training loss 66.38
2021-03-18 21:44:47,723 - INFO - joeynmt.training - EPOCH 87
2021-03-18 21:45:27,480 - INFO - joeynmt.training - Epoch  87: total training loss 65.81
2021-03-18 21:45:27,481 - INFO - joeynmt.training - EPOCH 88
2021-03-18 21:46:11,129 - INFO - joeynmt.training - Epoch  88: total training loss 65.08
2021-03-18 21:46:11,129 - INFO - joeynmt.training - EPOCH 89
2021-03-18 21:46:53,704 - INFO - joeynmt.training - Epoch  89: total training loss 64.30
2021-03-18 21:46:53,704 - INFO - joeynmt.training - EPOCH 90
2021-03-18 21:47:36,456 - INFO - joeynmt.training - Epoch  90: total training loss 64.23
2021-03-18 21:47:36,456 - INFO - joeynmt.training - EPOCH 91
2021-03-18 21:48:19,383 - INFO - joeynmt.training - Epoch  91: total training loss 63.59
2021-03-18 21:48:19,383 - INFO - joeynmt.training - EPOCH 92
2021-03-18 21:48:57,668 - INFO - joeynmt.training - Epoch  92: total training loss 62.96
2021-03-18 

## Con bpe 4000 pocas capas

In [82]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/con-random3/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/con-random3"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [83]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 22:11:07,364 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 22:11:07,452 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 22:11:07,753 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 22:11:33,095 - INFO - joeynmt.training - Epoch   1: total training loss 262.30
2021-03-18 22:11:33,095 - INFO - joeynmt.training - EPOCH 2
2021-03-18 22:11:54,968 - INFO - joeynmt.training - Epoch   2: total training loss 243.10
2021-03-18 22:11:54,969 - INFO - joeynmt.training - EPOCH 3
2021-03-18 22:12:17,303 - INFO - joeynmt.training - Epoch   3: total training loss 229.83
2021-03-18 22:12:17,304 - INFO - joeynmt.training - EPOCH 4
2021-03-18 22:12:35,656 - INFO - joeynmt.training - Epoch   4: total training loss 222.61
2021-03-18 22:12:35,657 - INFO - joeynmt.training - EP

2021-03-18 22:21:12,886 - INFO - joeynmt.training - 	Source:     porque cualquiera que se enaltece, será humillado y el que se humilla, será enaltecido
2021-03-18 22:21:12,886 - INFO - joeynmt.training - 	Reference:  ix pën jaty nëmyëjjaantsypëdaakëp, tsyuuk pëdaꞌaganëp jeꞌe a pën nëtsyuukpëdaakëp, jeꞌejëts mëj jaantsy pëdaꞌaganëp
2021-03-18 22:21:12,886 - INFO - joeynmt.training - 	Hypothesis: a ja ja ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy ajxy
2021-03-18 22:21:12,886 - INFO - joeynmt.training - Validation result (greedy) at epoch  27, step     1000: bleu:   0.07, loss: 124383.8906, ppl: 217.2589, duration: 23.4407s
2021-03-18 22:21:28,481 - INFO - joeynmt.training - Epoch  27: total training loss 194.68
2021-03-18 22:21:28,481 - INFO - joeynmt.training - EPOCH 28
2021-03-18 22:21:50,620 - INFO - 

2021-03-18 22:31:24,836 - INFO - joeynmt.training - Epoch  53: total training loss 172.41
2021-03-18 22:31:24,836 - INFO - joeynmt.training - EPOCH 54
2021-03-18 22:31:46,412 - INFO - joeynmt.training - Epoch  54: total training loss 171.90
2021-03-18 22:31:46,413 - INFO - joeynmt.training - EPOCH 55
2021-03-18 22:32:07,778 - INFO - joeynmt.training - Epoch  55: total training loss 171.32
2021-03-18 22:32:07,779 - INFO - joeynmt.training - EPOCH 56
2021-03-18 22:32:29,745 - INFO - joeynmt.training - Epoch  56: total training loss 170.78
2021-03-18 22:32:29,746 - INFO - joeynmt.training - EPOCH 57
2021-03-18 22:32:51,104 - INFO - joeynmt.training - Epoch  57: total training loss 170.28
2021-03-18 22:32:51,104 - INFO - joeynmt.training - EPOCH 58
2021-03-18 22:33:12,925 - INFO - joeynmt.training - Epoch  58: total training loss 169.66
2021-03-18 22:33:12,926 - INFO - joeynmt.training - EPOCH 59
2021-03-18 22:33:34,797 - INFO - joeynmt.training - Epoch  59: total training loss 169.19
2021

2021-03-18 22:41:24,469 - INFO - joeynmt.training - Epoch  79: total training loss 160.09
2021-03-18 22:41:24,470 - INFO - joeynmt.training - EPOCH 80
2021-03-18 22:41:47,050 - INFO - joeynmt.training - Epoch  80: total training loss 159.76
2021-03-18 22:41:47,051 - INFO - joeynmt.training - EPOCH 81
2021-03-18 22:42:04,695 - INFO - joeynmt.training - Epoch  81: total training loss 159.43
2021-03-18 22:42:04,695 - INFO - joeynmt.training - EPOCH 82
2021-03-18 22:42:26,040 - INFO - joeynmt.training - Epoch  82: total training loss 159.03
2021-03-18 22:42:26,040 - INFO - joeynmt.training - EPOCH 83
2021-03-18 22:42:47,937 - INFO - joeynmt.training - Epoch  83: total training loss 158.54
2021-03-18 22:42:47,937 - INFO - joeynmt.training - EPOCH 84
2021-03-18 22:43:10,277 - INFO - joeynmt.training - Epoch  84: total training loss 158.13
2021-03-18 22:43:10,277 - INFO - joeynmt.training - EPOCH 85
2021-03-18 22:43:32,934 - INFO - joeynmt.training - Epoch  85: total training loss 157.76
2021

### con bpe 4000 muchas capas

In [84]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/con-random4/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/con-random4"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [85]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-18 22:53:58,678 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-18 22:53:58,893 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-18 22:53:59,284 - INFO - joeynmt.data - building v

2021-03-18 22:54:50,743 - INFO - joeynmt.training - Epoch   1: total training loss 236.37
2021-03-18 22:54:50,743 - INFO - joeynmt.training - EPOCH 2
2021-03-18 22:55:37,102 - INFO - joeynmt.training - Epoch   2: total training loss 217.32
2021-03-18 22:55:37,103 - INFO - joeynmt.training - EPOCH 3
2021-03-18 22:56:25,064 - INFO - joeynmt.training - Epoch   3: total training loss 215.42
2021-03-18 22:56:25,065 - INFO - joeynmt.training - EPOCH 4
2021-03-18 22:57:11,832 - INFO - joeynmt.training - Epoch   4: total training loss 212.56
2021-03-18 22:57:11,833 - INFO - joeynmt.training - EPOCH 5
2021-03-18 22:57:50,590 - INFO - joeynmt.training - Epoch   5: total training loss 210.63
2021-03-18 22:57:50,591 - INFO - joeynmt.training - EPOCH 6
2021-03-18 22:58:37,016 - INFO - joeynmt.training - Epoch   6: total training loss 207.94
2021-03-18 22:58:37,017 - INFO - joeynmt.training - EPOCH 7
2021-03-18 22:59:22,771 - INFO - joeynmt.training - Epoch   7: total training loss 206.19
2021-03-18

2021-03-18 23:38:51,624 - INFO - joeynmt.training - Epoch  53: total training loss 109.13
2021-03-18 23:38:51,625 - INFO - joeynmt.training - EPOCH 54
2021-03-18 23:39:39,247 - INFO - joeynmt.training - Epoch  54: total training loss 108.11
2021-03-18 23:39:39,248 - INFO - joeynmt.training - EPOCH 55
2021-03-18 23:40:29,968 - INFO - joeynmt.training - Epoch  55: total training loss 107.02
2021-03-18 23:40:29,969 - INFO - joeynmt.training - EPOCH 56
2021-03-18 23:41:19,757 - INFO - joeynmt.training - Epoch  56: total training loss 104.84
2021-03-18 23:41:19,757 - INFO - joeynmt.training - EPOCH 57
2021-03-18 23:42:10,535 - INFO - joeynmt.training - Epoch  57: total training loss 104.18
2021-03-18 23:42:10,536 - INFO - joeynmt.training - EPOCH 58
2021-03-18 23:42:58,506 - INFO - joeynmt.training - Epoch  58: total training loss 102.44
2021-03-18 23:42:58,507 - INFO - joeynmt.training - EPOCH 59
2021-03-18 23:43:49,665 - INFO - joeynmt.training - Epoch  59: total training loss 101.10
2021

2021-03-19 00:03:09,229 - INFO - joeynmt.training - Epoch  80: total training loss 79.15
2021-03-19 00:03:09,229 - INFO - joeynmt.training - EPOCH 81
2021-03-19 00:03:59,338 - INFO - joeynmt.training - Epoch  81: total training loss 78.31
2021-03-19 00:03:59,339 - INFO - joeynmt.training - EPOCH 82
2021-03-19 00:04:48,426 - INFO - joeynmt.training - Epoch  82: total training loss 77.59
2021-03-19 00:04:48,426 - INFO - joeynmt.training - EPOCH 83
2021-03-19 00:05:30,372 - INFO - joeynmt.training - Epoch  83: total training loss 76.69
2021-03-19 00:05:30,372 - INFO - joeynmt.training - EPOCH 84
2021-03-19 00:06:17,820 - INFO - joeynmt.training - Epoch  84: total training loss 75.57
2021-03-19 00:06:17,820 - INFO - joeynmt.training - EPOCH 85
2021-03-19 00:07:05,254 - INFO - joeynmt.training - Epoch  85: total training loss 74.88
2021-03-19 00:07:05,254 - INFO - joeynmt.training - EPOCH 86
2021-03-19 00:07:52,640 - INFO - joeynmt.training - Epoch  86: total training loss 74.23
2021-03-19 

## con 70 longitud 4000bpe muchas capas

In [86]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/con-random5/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/con-random5"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [87]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 00:25:48,517 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 00:25:48,640 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 00:25:49,068 - INFO - joeynmt.data - building v

2021-03-19 00:26:57,726 - INFO - joeynmt.training - Epoch   1: total training loss 265.61
2021-03-19 00:26:57,727 - INFO - joeynmt.training - EPOCH 2
2021-03-19 00:28:01,869 - INFO - joeynmt.training - Epoch   2: total training loss 246.38
2021-03-19 00:28:01,869 - INFO - joeynmt.training - EPOCH 3
2021-03-19 00:29:04,536 - INFO - joeynmt.training - Epoch   3: total training loss 244.00
2021-03-19 00:29:04,536 - INFO - joeynmt.training - EPOCH 4
2021-03-19 00:30:01,947 - INFO - joeynmt.training - Epoch   4: total training loss 241.83
2021-03-19 00:30:01,947 - INFO - joeynmt.training - EPOCH 5
2021-03-19 00:31:05,096 - INFO - joeynmt.training - Epoch   5: total training loss 239.22
2021-03-19 00:31:05,096 - INFO - joeynmt.training - EPOCH 6
2021-03-19 00:32:09,333 - INFO - joeynmt.training - Epoch   6: total training loss 237.36
2021-03-19 00:32:09,334 - INFO - joeynmt.training - EPOCH 7
2021-03-19 00:33:10,397 - INFO - joeynmt.training - Epoch   7: total training loss 234.49
2021-03-19

2021-03-19 00:54:10,084 - INFO - joeynmt.training - 	Hypothesis: a ku ja yaꞌay ajxy ënajty të t�� të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të të
2021-03-19 00:54:10,084 - INFO - joeynmt.training - Validation result (greedy) at epoch  24, step     1000: bleu:   0.38, loss: 105258.1172, ppl:  94.9805, duration: 87.2451s
2021-03-19 00:54:58,597 - INFO - joeynmt.training - Epoch  24: total training loss 183.43
2021-03-19 00:54:58,597 - INFO - joeynmt.training - EPOCH 25
2021-03-19 00:56:04,654 - INFO - joeynmt.training - Epoch  25: total training loss 181.02
2021-03-19 00:56:04,655 - INFO - joeynmt.training - EPOCH 26
2021-03-19 01:03:36,256 - INFO - joeynmt.training - Epoch  32: total training loss 165.67
2021-03-19 01:03:36,256 - INFO - joeynmt.training - EPOCH 33
2021-03-19 01:04:39,923 - INFO - joeynmt.training - Epoch  33: total training loss 163.1

2021-03-19 01:27:30,550 - INFO - joeynmt.training - Epoch  51: total training loss 129.20
2021-03-19 01:27:30,550 - INFO - joeynmt.training - EPOCH 52
2021-03-19 01:28:37,420 - INFO - joeynmt.training - Epoch  52: total training loss 127.84
2021-03-19 01:28:37,421 - INFO - joeynmt.training - EPOCH 53
2021-03-19 01:29:42,905 - INFO - joeynmt.training - Epoch  53: total training loss 126.03
2021-03-19 01:29:42,905 - INFO - joeynmt.training - EPOCH 54
2021-03-19 01:30:46,404 - INFO - joeynmt.training - Epoch  54: total training loss 125.24
2021-03-19 01:30:46,405 - INFO - joeynmt.training - EPOCH 55
2021-03-19 01:31:55,192 - INFO - joeynmt.training - Epoch  55: total training loss 123.22
2021-03-19 01:31:55,192 - INFO - joeynmt.training - EPOCH 56
2021-03-19 01:33:01,596 - INFO - joeynmt.training - Epoch  56: total training loss 121.77
2021-03-19 01:33:01,597 - INFO - joeynmt.training - EPOCH 57
2021-03-19 01:34:06,670 - INFO - joeynmt.training - Epoch  57: total training loss 120.16
2021

2021-03-19 01:58:20,993 - INFO - joeynmt.training - Epoch  76: total training loss 97.67
2021-03-19 01:58:20,993 - INFO - joeynmt.training - EPOCH 77
2021-03-19 01:59:30,118 - INFO - joeynmt.training - Epoch  77: total training loss 96.87
2021-03-19 01:59:30,118 - INFO - joeynmt.training - EPOCH 78
2021-03-19 02:00:40,722 - INFO - joeynmt.training - Epoch  78: total training loss 95.89
2021-03-19 02:00:40,723 - INFO - joeynmt.training - EPOCH 79
2021-03-19 02:01:50,920 - INFO - joeynmt.training - Epoch  79: total training loss 95.11
2021-03-19 02:01:50,920 - INFO - joeynmt.training - EPOCH 80
2021-03-19 02:02:54,160 - INFO - joeynmt.training - Epoch  80: total training loss 94.04
2021-03-19 02:02:54,160 - INFO - joeynmt.training - EPOCH 81
2021-03-19 02:04:04,732 - INFO - joeynmt.training - Epoch  81: total training loss 93.24
2021-03-19 02:04:04,733 - INFO - joeynmt.training - EPOCH 82
2021-03-19 02:05:48,879 - INFO - joeynmt.training - Example #0
2021-03-19 02:05:48,880 - INFO - joey

2021-03-19 02:29:56,086 - INFO - joeynmt.prediction -  dev bleu[13a]:   5.83 [Beam search decoding with beam size = 3 and alpha = 1.0]
2021-03-19 02:29:56,088 - INFO - joeynmt.prediction - Translations saved to: models/con-random5/00004000.hyps.dev
2021-03-19 02:29:56,088 - INFO - joeynmt.prediction - Decoding on test set (/data/joeynmt/mir.esmir/test.bpe.mir)...
2021-03-19 02:32:40,979 - INFO - joeynmt.prediction - test bleu[13a]:   5.56 [Beam search decoding with beam size = 3 and alpha = 1.0]
2021-03-19 02:32:40,981 - INFO - joeynmt.prediction - Translations saved to: models/con-random5/00004000.hyps.test


# Ahora en direccion opuesto donde source = "mir" y target="es"   con random

## Con bpe 4000 muchas capas

In [96]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves1/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves1"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [97]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 06:03:50,534 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 06:03:50,616 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 06:03:51,103 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 06:05:04,981 - INFO - joeynmt.training - Epoch   1: total training loss 260.25
2021-03-19 06:05:04,982 - INFO - joeynmt.training - EPOCH 2
2021-03-19 06:06:18,877 - INFO - joeynmt.training - Epoch   2: total training loss 236.67
2021-03-19 06:06:18,877 - INFO - joeynmt.training - EPOCH 3
2021-03-19 06:07:27,217 - INFO - joeynmt.training - Epoch   3: total training loss 234.39
2021-03-19 06:07:27,219 - INFO - joeynmt.training - EPOCH 4
2021-03-19 06:08:32,997 - INFO - joeynmt.training - Epoch   4: total training loss 232.04
2021-03-19 06:08:32,997 - INFO - joeynmt.training - EP

2021-03-19 06:31:53,885 - INFO - joeynmt.training - Epoch  24: total training loss 177.44
2021-03-19 06:31:53,885 - INFO - joeynmt.training - EPOCH 25
2021-03-19 06:32:57,710 - INFO - joeynmt.training - Epoch  25: total training loss 175.81
2021-03-19 06:32:57,711 - INFO - joeynmt.training - EPOCH 26
2021-03-19 06:34:00,973 - INFO - joeynmt.training - Epoch  26: total training loss 173.32
2021-03-19 06:34:00,973 - INFO - joeynmt.training - EPOCH 27
2021-03-19 06:35:00,938 - INFO - joeynmt.training - Epoch  27: total training loss 171.21
2021-03-19 06:35:00,938 - INFO - joeynmt.training - EPOCH 28
2021-03-19 06:36:08,005 - INFO - joeynmt.training - Epoch  28: total training loss 168.89
2021-03-19 06:36:08,005 - INFO - joeynmt.training - EPOCH 29
2021-03-19 06:37:09,924 - INFO - joeynmt.training - Epoch  29: total training loss 166.48
2021-03-19 06:37:09,925 - INFO - joeynmt.training - EPOCH 30
2021-03-19 06:38:10,361 - INFO - joeynmt.training - Epoch  30: total training loss 164.81
2021

2021-03-19 07:01:02,118 - INFO - joeynmt.training - Epoch  49: total training loss 130.00
2021-03-19 07:01:02,118 - INFO - joeynmt.training - EPOCH 50
2021-03-19 07:02:07,315 - INFO - joeynmt.training - Epoch  50: total training loss 128.95
2021-03-19 07:02:07,315 - INFO - joeynmt.training - EPOCH 51
2021-03-19 07:03:07,391 - INFO - joeynmt.training - Epoch  51: total training loss 126.90
2021-03-19 07:03:07,391 - INFO - joeynmt.training - EPOCH 52
2021-03-19 07:04:13,265 - INFO - joeynmt.training - Epoch  52: total training loss 125.76
2021-03-19 07:04:13,265 - INFO - joeynmt.training - EPOCH 53
2021-03-19 07:05:17,625 - INFO - joeynmt.training - Epoch  53: total training loss 124.03
2021-03-19 07:05:17,625 - INFO - joeynmt.training - EPOCH 54
2021-03-19 07:06:17,014 - INFO - joeynmt.training - Epoch  54: total training loss 123.75
2021-03-19 07:06:17,014 - INFO - joeynmt.training - EPOCH 55
2021-03-19 07:07:21,137 - INFO - joeynmt.training - Epoch  55: total training loss 121.43
2021

2021-03-19 07:31:42,140 - INFO - joeynmt.training - Epoch  75: total training loss 96.79
2021-03-19 07:31:42,140 - INFO - joeynmt.training - EPOCH 76
2021-03-19 07:32:47,892 - INFO - joeynmt.training - Epoch  76: total training loss 95.51
2021-03-19 07:32:47,893 - INFO - joeynmt.training - EPOCH 77
2021-03-19 07:33:55,051 - INFO - joeynmt.training - Epoch  77: total training loss 94.52
2021-03-19 07:33:55,053 - INFO - joeynmt.training - EPOCH 78
2021-03-19 07:34:53,932 - INFO - joeynmt.training - Epoch  78: total training loss 93.76
2021-03-19 07:34:53,932 - INFO - joeynmt.training - EPOCH 79
2021-03-19 07:36:01,176 - INFO - joeynmt.training - Epoch  79: total training loss 92.45
2021-03-19 07:36:01,177 - INFO - joeynmt.training - EPOCH 80
2021-03-19 07:37:08,644 - INFO - joeynmt.training - Epoch  80: total training loss 91.69
2021-03-19 07:37:08,645 - INFO - joeynmt.training - EPOCH 81
2021-03-19 07:38:15,235 - INFO - joeynmt.training - Epoch  81: total training loss 90.79
2021-03-19 

2021-03-19 08:02:19,863 - INFO - joeynmt.prediction - Decoding on dev set (/data/joeynmt/mir.mires/dev.bpe.es)...
2021-03-19 08:03:12,556 - INFO - joeynmt.prediction -  dev bleu[13a]:   3.64 [Beam search decoding with beam size = 3 and alpha = 1.0]
2021-03-19 08:03:12,557 - INFO - joeynmt.prediction - Translations saved to: models/alreves1/00003000.hyps.dev
2021-03-19 08:03:12,557 - INFO - joeynmt.prediction - Decoding on test set (/data/joeynmt/mir.mires/test.bpe.es)...
2021-03-19 08:05:13,110 - INFO - joeynmt.prediction - test bleu[13a]:   3.52 [Beam search decoding with beam size = 3 and alpha = 1.0]
2021-03-19 08:05:13,111 - INFO - joeynmt.prediction - Translations saved to: models/alreves1/00003000.hyps.test


In [105]:
! cd joeynmt; /ve/bin/python3 -m joeynmt translate models/alreves1/config.yaml < models/alreves1/frases_prueba.txt

2021-03-19 16:09:49,903 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/ve/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 relea

### con50 longitud

In [106]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves2/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves2"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [107]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 16:19:15,636 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 16:19:15,707 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 16:19:16,058 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 16:20:12,775 - INFO - joeynmt.training - Epoch   1: total training loss 231.40
2021-03-19 16:20:12,775 - INFO - joeynmt.training - EPOCH 2
2021-03-19 16:20:55,144 - INFO - joeynmt.training - Epoch   2: total training loss 208.24
2021-03-19 16:20:55,144 - INFO - joeynmt.training - EPOCH 3
2021-03-19 16:21:41,555 - INFO - joeynmt.training - Epoch   3: total training loss 206.35
2021-03-19 16:21:41,555 - INFO - joeynmt.training - EPOCH 4
2021-03-19 16:22:28,110 - INFO - joeynmt.training - Epoch   4: total training loss 204.27
2021-03-19 16:22:28,116 - INFO - joeynmt.training - EP

2021-03-19 16:42:10,691 - INFO - joeynmt.training - Epoch  27: total training loss 154.06
2021-03-19 16:42:10,692 - INFO - joeynmt.training - EPOCH 28
2021-03-19 16:43:00,954 - INFO - joeynmt.training - Epoch  28: total training loss 152.34
2021-03-19 16:43:00,954 - INFO - joeynmt.training - EPOCH 29
2021-03-19 16:43:49,048 - INFO - joeynmt.training - Epoch  29: total training loss 149.96
2021-03-19 16:43:49,048 - INFO - joeynmt.training - EPOCH 30
2021-03-19 16:44:29,727 - INFO - joeynmt.training - Epoch  30: total training loss 148.19
2021-03-19 16:44:29,729 - INFO - joeynmt.training - EPOCH 31
2021-03-19 16:45:18,163 - INFO - joeynmt.training - Epoch  31: total training loss 146.28
2021-03-19 16:45:18,164 - INFO - joeynmt.training - EPOCH 32
2021-03-19 16:46:08,303 - INFO - joeynmt.training - Epoch  32: total training loss 144.01
2021-03-19 16:46:08,305 - INFO - joeynmt.training - EPOCH 33
2021-03-19 16:46:53,929 - INFO - joeynmt.training - Epoch  33: total training loss 141.89
2021

2021-03-19 17:05:29,624 - INFO - joeynmt.training - Epoch  54: total training loss 107.53
2021-03-19 17:05:29,624 - INFO - joeynmt.training - EPOCH 55
2021-03-19 17:06:20,169 - INFO - joeynmt.training - Epoch  55: total training loss 106.60
2021-03-19 17:06:20,172 - INFO - joeynmt.training - EPOCH 56
2021-03-19 17:07:08,262 - INFO - joeynmt.training - Epoch  56: total training loss 104.73
2021-03-19 17:07:08,262 - INFO - joeynmt.training - EPOCH 57
2021-03-19 17:07:54,455 - INFO - joeynmt.training - Epoch  57: total training loss 103.77
2021-03-19 17:07:54,456 - INFO - joeynmt.training - EPOCH 58
2021-03-19 17:08:36,011 - INFO - joeynmt.training - Epoch  58: total training loss 102.52
2021-03-19 17:08:36,011 - INFO - joeynmt.training - EPOCH 59
2021-03-19 17:09:30,007 - INFO - joeynmt.training - Epoch  59: total training loss 100.95
2021-03-19 17:09:30,007 - INFO - joeynmt.training - EPOCH 60
2021-03-19 17:10:24,336 - INFO - joeynmt.training - Epoch  60: total training loss 99.44
2021-

2021-03-19 17:30:53,413 - INFO - joeynmt.training - Epoch  83: total training loss 74.84
2021-03-19 17:30:53,413 - INFO - joeynmt.training - EPOCH 84
2021-03-19 17:31:40,746 - INFO - joeynmt.training - Epoch  84: total training loss 73.66
2021-03-19 17:31:40,746 - INFO - joeynmt.training - EPOCH 85
2021-03-19 17:32:28,748 - INFO - joeynmt.training - Epoch  85: total training loss 72.95
2021-03-19 17:32:28,748 - INFO - joeynmt.training - EPOCH 86
2021-03-19 17:33:16,615 - INFO - joeynmt.training - Epoch  86: total training loss 72.24
2021-03-19 17:33:16,615 - INFO - joeynmt.training - EPOCH 87
2021-03-19 17:34:05,046 - INFO - joeynmt.training - Epoch  87: total training loss 71.39
2021-03-19 17:34:05,046 - INFO - joeynmt.training - EPOCH 88
2021-03-19 17:34:54,054 - INFO - joeynmt.training - Epoch  88: total training loss 70.51
2021-03-19 17:34:54,055 - INFO - joeynmt.training - EPOCH 89
2021-03-19 17:35:37,964 - INFO - joeynmt.training - Epoch  89: total training loss 69.80
2021-03-19 

### Con bpe 4000 pocas capas MIR-ES con random

In [108]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves3/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves3"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [109]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 17:53:48,491 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 17:53:48,546 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 17:53:48,795 - INFO - joeynmt.data - building v

2021-03-19 17:54:07,878 - INFO - joeynmt.training - Epoch   1: total training loss 262.19
2021-03-19 17:54:07,879 - INFO - joeynmt.training - EPOCH 2
2021-03-19 17:54:24,298 - INFO - joeynmt.training - Epoch   2: total training loss 240.10
2021-03-19 17:54:24,299 - INFO - joeynmt.training - EPOCH 3
2021-03-19 17:54:41,886 - INFO - joeynmt.training - Epoch   3: total training loss 223.90
2021-03-19 17:54:41,887 - INFO - joeynmt.training - EPOCH 4
2021-03-19 17:54:59,357 - INFO - joeynmt.training - Epoch   4: total training loss 214.73
2021-03-19 17:54:59,360 - INFO - joeynmt.training - EPOCH 5
2021-03-19 17:55:15,976 - INFO - joeynmt.training - Epoch   5: total training loss 210.36
2021-03-19 17:55:15,977 - INFO - joeynmt.training - EPOCH 6
2021-03-19 17:55:32,266 - INFO - joeynmt.training - Epoch   6: total training loss 208.49
2021-03-19 17:55:32,267 - INFO - joeynmt.training - EPOCH 7
2021-03-19 17:55:49,674 - INFO - joeynmt.training - Epoch   7: total training loss 207.79
2021-03-19

2021-03-19 18:01:43,173 - INFO - joeynmt.training - Epoch  27: total training loss 186.72
2021-03-19 18:01:43,173 - INFO - joeynmt.training - EPOCH 28
2021-03-19 18:01:59,996 - INFO - joeynmt.training - Epoch  28: total training loss 185.31
2021-03-19 18:01:59,996 - INFO - joeynmt.training - EPOCH 29
2021-03-19 18:02:17,175 - INFO - joeynmt.training - Epoch  29: total training loss 184.08
2021-03-19 18:02:17,175 - INFO - joeynmt.training - EPOCH 30
2021-03-19 18:02:33,368 - INFO - joeynmt.training - Epoch  30: total training loss 182.85
2021-03-19 18:02:33,369 - INFO - joeynmt.training - EPOCH 31
2021-03-19 18:02:49,534 - INFO - joeynmt.training - Epoch  31: total training loss 181.80
2021-03-19 18:02:49,535 - INFO - joeynmt.training - EPOCH 32
2021-03-19 18:03:01,313 - INFO - joeynmt.training - Epoch  32: total training loss 180.85
2021-03-19 18:03:01,313 - INFO - joeynmt.training - EPOCH 33
2021-03-19 18:03:13,837 - INFO - joeynmt.training - Epoch  33: total training loss 180.06
2021

2021-03-19 18:09:07,539 - INFO - joeynmt.training - Epoch  53: total training loss 167.78
2021-03-19 18:09:07,539 - INFO - joeynmt.training - EPOCH 54
2021-03-19 18:09:24,291 - INFO - joeynmt.training - Epoch  54: total training loss 167.25
2021-03-19 18:09:24,292 - INFO - joeynmt.training - EPOCH 55
2021-03-19 18:09:40,370 - INFO - joeynmt.training - Epoch  55: total training loss 166.75
2021-03-19 18:09:40,370 - INFO - joeynmt.training - EPOCH 56
2021-03-19 18:09:57,686 - INFO - joeynmt.training - Epoch  56: total training loss 166.23
2021-03-19 18:09:57,687 - INFO - joeynmt.training - EPOCH 57
2021-03-19 18:10:14,202 - INFO - joeynmt.training - Epoch  57: total training loss 165.72
2021-03-19 18:10:14,203 - INFO - joeynmt.training - EPOCH 58
2021-03-19 18:10:30,614 - INFO - joeynmt.training - Epoch  58: total training loss 165.24
2021-03-19 18:10:30,614 - INFO - joeynmt.training - EPOCH 59
2021-03-19 18:10:47,115 - INFO - joeynmt.training - Epoch  59: total training loss 164.89
2021

2021-03-19 18:16:36,684 - INFO - joeynmt.training - Validation result (greedy) at epoch  79, step     3000: bleu:   0.33, loss: 88887.0859, ppl: 107.2400, duration: 16.2891s
2021-03-19 18:16:37,591 - INFO - joeynmt.training - Epoch  79: total training loss 155.46
2021-03-19 18:16:37,591 - INFO - joeynmt.training - EPOCH 80
2021-03-19 18:16:54,373 - INFO - joeynmt.training - Epoch  80: total training loss 154.97
2021-03-19 18:16:54,374 - INFO - joeynmt.training - EPOCH 81
2021-03-19 18:17:11,941 - INFO - joeynmt.training - Epoch  81: total training loss 154.75
2021-03-19 18:17:11,942 - INFO - joeynmt.training - EPOCH 82
2021-03-19 18:17:30,018 - INFO - joeynmt.training - Epoch  82: total training loss 154.37
2021-03-19 18:17:30,018 - INFO - joeynmt.training - EPOCH 83
2021-03-19 18:17:47,446 - INFO - joeynmt.training - Epoch  83: total training loss 153.75
2021-03-19 18:17:47,446 - INFO - joeynmt.training - EPOCH 84
2021-03-19 18:18:04,671 - INFO - joeynmt.training - Epoch  84: total tr

### con bpe 2000

#### con pocas capas

In [118]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves4/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves4"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [119]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 18:33:12,035 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 18:33:12,111 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 18:33:12,585 - INFO - joeynmt.data - building v

2021-03-19 18:33:26,842 - INFO - joeynmt.training - Epoch   1: total training loss 219.74
2021-03-19 18:33:26,842 - INFO - joeynmt.training - EPOCH 2
2021-03-19 18:33:38,397 - INFO - joeynmt.training - Epoch   2: total training loss 201.90
2021-03-19 18:33:38,399 - INFO - joeynmt.training - EPOCH 3
2021-03-19 18:33:49,490 - INFO - joeynmt.training - Epoch   3: total training loss 190.27
2021-03-19 18:33:49,490 - INFO - joeynmt.training - EPOCH 4
2021-03-19 18:34:00,791 - INFO - joeynmt.training - Epoch   4: total training loss 184.49
2021-03-19 18:34:00,792 - INFO - joeynmt.training - EPOCH 5
2021-03-19 18:34:12,688 - INFO - joeynmt.training - Epoch   5: total training loss 182.08
2021-03-19 18:34:12,688 - INFO - joeynmt.training - EPOCH 6
2021-03-19 18:34:24,607 - INFO - joeynmt.training - Epoch   6: total training loss 181.28
2021-03-19 18:34:24,607 - INFO - joeynmt.training - EPOCH 7
2021-03-19 18:34:35,492 - INFO - joeynmt.training - Epoch   7: total training loss 180.77
2021-03-19

2021-03-19 18:39:19,878 - INFO - joeynmt.training - Epoch  29: total training loss 163.10
2021-03-19 18:39:19,878 - INFO - joeynmt.training - EPOCH 30
2021-03-19 18:39:27,619 - INFO - joeynmt.training - Epoch  30: total training loss 162.17
2021-03-19 18:39:27,620 - INFO - joeynmt.training - EPOCH 31
2021-03-19 18:39:37,989 - INFO - joeynmt.training - Epoch  31: total training loss 161.27
2021-03-19 18:39:37,989 - INFO - joeynmt.training - EPOCH 32
2021-03-19 18:39:50,422 - INFO - joeynmt.training - Epoch  32: total training loss 160.63
2021-03-19 18:39:50,422 - INFO - joeynmt.training - EPOCH 33
2021-03-19 18:40:01,992 - INFO - joeynmt.training - Epoch  33: total training loss 159.82
2021-03-19 18:40:01,993 - INFO - joeynmt.training - EPOCH 34
2021-03-19 18:40:14,102 - INFO - joeynmt.training - Epoch  34: total training loss 159.19
2021-03-19 18:40:14,102 - INFO - joeynmt.training - EPOCH 35
2021-03-19 18:40:26,084 - INFO - joeynmt.training - Epoch  35: total training loss 158.46
2021

2021-03-19 18:45:19,089 - INFO - joeynmt.training - 	Hypothesis: y le dijo no que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que que no no no no no no no no no
2021-03-19 18:45:19,089 - INFO - joeynmt.training - Validation result (greedy) at epoch  58, step     2000: bleu:   0.15, loss: 101790.3750, ppl:  98.4242, duration: 16.7818s
2021-03-19 18:45:28,821 - INFO - joeynmt.training - Epoch  58: total training loss 147.26
2021-03-19 18:45:28,821 - INFO - joeynmt.training - EPOCH 59
2021-03-19 18:45:40,754 - INFO - joeynmt.training - Epoch  59: total training loss 146.79
2021-03-19 18:45:40,754 - INFO - joeynmt.training - EPOCH 60
2021-03-19 18:45:52,436 - INFO - joeynmt.training - Epoch  60: total training loss 146.44
2021-03-19 18:45:52,437 - INFO - joeynmt.training - EPOCH 61
2021-03-19 18:46:03,771 - INFO - joeynmt.training - Epoch  61: total training loss 146.10
2021-03-19 18:46:03,771 - INF

2021-03-19 18:51:35,749 - INFO - joeynmt.training - Epoch  86: total training loss 136.78
2021-03-19 18:51:35,749 - INFO - joeynmt.training - EPOCH 87
2021-03-19 18:51:47,908 - INFO - joeynmt.training - Epoch  87: total training loss 136.30
2021-03-19 18:51:47,908 - INFO - joeynmt.training - EPOCH 88
2021-03-19 18:52:00,173 - INFO - joeynmt.training - Epoch  88: total training loss 136.10
2021-03-19 18:52:00,173 - INFO - joeynmt.training - EPOCH 89
2021-03-19 18:52:12,475 - INFO - joeynmt.training - Epoch  89: total training loss 135.59
2021-03-19 18:52:12,476 - INFO - joeynmt.training - EPOCH 90
2021-03-19 18:52:24,747 - INFO - joeynmt.training - Epoch  90: total training loss 135.42
2021-03-19 18:52:24,747 - INFO - joeynmt.training - EPOCH 91
2021-03-19 18:52:37,352 - INFO - joeynmt.training - Epoch  91: total training loss 134.95
2021-03-19 18:52:37,354 - INFO - joeynmt.training - EPOCH 92
2021-03-19 18:52:45,845 - INFO - joeynmt.training - Epoch  92: total training loss 134.72
2021

## con bpe 2000 muchas capas

In [123]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves5/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves5"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [124]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 19:39:06,166 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 19:39:06,212 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 19:39:06,494 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 19:39:47,334 - INFO - joeynmt.training - Epoch   1: total training loss 197.50
2021-03-19 19:39:47,334 - INFO - joeynmt.training - EPOCH 2
2021-03-19 19:40:31,885 - INFO - joeynmt.training - Epoch   2: total training loss 181.69
2021-03-19 19:40:31,886 - INFO - joeynmt.training - EPOCH 3
2021-03-19 19:41:06,929 - INFO - joeynmt.training - Epoch   3: total training loss 180.33
2021-03-19 19:41:06,930 - INFO - joeynmt.training - EPOCH 4
2021-03-19 19:41:47,663 - INFO - joeynmt.training - Epoch   4: total training loss 178.17
2021-03-19 19:41:47,664 - INFO - joeynmt.training - EP

2021-03-19 19:58:57,131 - INFO - joeynmt.training - Epoch  29: total training loss 118.76
2021-03-19 19:58:57,131 - INFO - joeynmt.training - EPOCH 30
2021-03-19 19:59:28,406 - INFO - joeynmt.training - Epoch  30: total training loss 116.87
2021-03-19 19:59:28,407 - INFO - joeynmt.training - EPOCH 31
2021-03-19 20:00:08,223 - INFO - joeynmt.training - Epoch  31: total training loss 116.19
2021-03-19 20:00:08,224 - INFO - joeynmt.training - EPOCH 32
2021-03-19 20:00:48,426 - INFO - joeynmt.training - Epoch  32: total training loss 113.71
2021-03-19 20:00:48,426 - INFO - joeynmt.training - EPOCH 33
2021-03-19 20:01:27,292 - INFO - joeynmt.training - Epoch  33: total training loss 112.14
2021-03-19 20:01:27,292 - INFO - joeynmt.training - EPOCH 34
2021-03-19 20:01:58,772 - INFO - joeynmt.training - Epoch  34: total training loss 110.06
2021-03-19 20:01:58,773 - INFO - joeynmt.training - EPOCH 35
2021-03-19 20:02:36,963 - INFO - joeynmt.training - Epoch  35: total training loss 109.30
2021

2021-03-19 20:18:52,603 - INFO - joeynmt.training - Validation result (greedy) at epoch  58, step     2000: bleu:   2.23, loss: 83043.1172, ppl:  42.2690, duration: 65.8797s
2021-03-19 20:19:27,672 - INFO - joeynmt.training - Epoch  58: total training loss 82.75
2021-03-19 20:19:27,673 - INFO - joeynmt.training - EPOCH 59
2021-03-19 20:20:08,075 - INFO - joeynmt.training - Epoch  59: total training loss 82.18
2021-03-19 20:20:08,078 - INFO - joeynmt.training - EPOCH 60
2021-03-19 20:20:40,519 - INFO - joeynmt.training - Epoch  60: total training loss 81.42
2021-03-19 20:20:40,520 - INFO - joeynmt.training - EPOCH 61
2021-03-19 20:21:24,727 - INFO - joeynmt.training - Epoch  61: total training loss 80.37
2021-03-19 20:21:24,729 - INFO - joeynmt.training - EPOCH 62
2021-03-19 20:22:10,458 - INFO - joeynmt.training - Epoch  62: total training loss 79.18
2021-03-19 20:22:10,458 - INFO - joeynmt.training - EPOCH 63
2021-03-19 20:22:53,549 - INFO - joeynmt.training - Epoch  63: total trainin

2021-03-19 20:40:58,593 - INFO - joeynmt.training - Epoch  86: total training loss 62.15
2021-03-19 20:40:58,594 - INFO - joeynmt.training - EPOCH 87
2021-03-19 20:41:35,118 - INFO - joeynmt.training - Epoch  87: total training loss 61.79
2021-03-19 20:41:35,119 - INFO - joeynmt.training - EPOCH 88
2021-03-19 20:42:15,573 - INFO - joeynmt.training - Epoch  88: total training loss 60.96
2021-03-19 20:42:15,573 - INFO - joeynmt.training - EPOCH 89
2021-03-19 20:42:59,983 - INFO - joeynmt.training - Epoch  89: total training loss 60.47
2021-03-19 20:42:59,983 - INFO - joeynmt.training - EPOCH 90
2021-03-19 20:43:44,982 - INFO - joeynmt.training - Epoch  90: total training loss 59.95
2021-03-19 20:43:44,982 - INFO - joeynmt.training - EPOCH 91
2021-03-19 20:44:31,474 - INFO - joeynmt.training - Epoch  91: total training loss 59.50
2021-03-19 20:44:31,474 - INFO - joeynmt.training - EPOCH 92
2021-03-19 20:45:18,042 - INFO - joeynmt.training - Epoch  92: total training loss 58.84
2021-03-19 

## SIN RANDOM MIR-ES 

### CON BPE 4000 POCAS CAPAS

In [133]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves-sin1/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves-sin1"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [134]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 21:08:27,125 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 21:08:27,186 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 21:08:27,720 - INFO - joeynmt.data - building v

2021-03-19 21:08:51,313 - INFO - joeynmt.training - Epoch   1: total training loss 254.52
2021-03-19 21:08:51,313 - INFO - joeynmt.training - EPOCH 2
2021-03-19 21:09:12,696 - INFO - joeynmt.training - Epoch   2: total training loss 233.89
2021-03-19 21:09:12,696 - INFO - joeynmt.training - EPOCH 3
2021-03-19 21:09:33,205 - INFO - joeynmt.training - Epoch   3: total training loss 218.63
2021-03-19 21:09:33,206 - INFO - joeynmt.training - EPOCH 4
2021-03-19 21:09:48,910 - INFO - joeynmt.training - Epoch   4: total training loss 209.89
2021-03-19 21:09:48,910 - INFO - joeynmt.training - EPOCH 5
2021-03-19 21:10:03,082 - INFO - joeynmt.training - Epoch   5: total training loss 205.71
2021-03-19 21:10:03,082 - INFO - joeynmt.training - EPOCH 6
2021-03-19 21:10:21,792 - INFO - joeynmt.training - Epoch   6: total training loss 204.18
2021-03-19 21:10:21,793 - INFO - joeynmt.training - EPOCH 7
2021-03-19 21:10:39,811 - INFO - joeynmt.training - Epoch   7: total training loss 203.42
2021-03-19

2021-03-19 21:17:36,763 - INFO - joeynmt.training - Epoch  28: total training loss 184.48
2021-03-19 21:17:36,764 - INFO - joeynmt.training - EPOCH 29
2021-03-19 21:17:57,381 - INFO - joeynmt.training - Epoch  29: total training loss 183.06
2021-03-19 21:17:57,382 - INFO - joeynmt.training - EPOCH 30
2021-03-19 21:18:17,675 - INFO - joeynmt.training - Epoch  30: total training loss 181.71
2021-03-19 21:18:17,675 - INFO - joeynmt.training - EPOCH 31
2021-03-19 21:18:37,669 - INFO - joeynmt.training - Epoch  31: total training loss 180.48
2021-03-19 21:18:37,670 - INFO - joeynmt.training - EPOCH 32
2021-03-19 21:18:53,870 - INFO - joeynmt.training - Epoch  32: total training loss 179.38
2021-03-19 21:18:53,871 - INFO - joeynmt.training - EPOCH 33
2021-03-19 21:19:11,010 - INFO - joeynmt.training - Epoch  33: total training loss 178.45
2021-03-19 21:19:11,011 - INFO - joeynmt.training - EPOCH 34
2021-03-19 21:19:32,618 - INFO - joeynmt.training - Epoch  34: total training loss 177.46
2021

2021-03-19 21:26:58,709 - INFO - joeynmt.training - Epoch  55: total training loss 164.73
2021-03-19 21:26:58,710 - INFO - joeynmt.training - EPOCH 56
2021-03-19 21:27:12,570 - INFO - joeynmt.training - Epoch  56: total training loss 164.30
2021-03-19 21:27:12,570 - INFO - joeynmt.training - EPOCH 57
2021-03-19 21:27:30,398 - INFO - joeynmt.training - Epoch  57: total training loss 163.76
2021-03-19 21:27:30,398 - INFO - joeynmt.training - EPOCH 58
2021-03-19 21:27:51,334 - INFO - joeynmt.training - Epoch  58: total training loss 163.19
2021-03-19 21:27:51,334 - INFO - joeynmt.training - EPOCH 59
2021-03-19 21:28:11,646 - INFO - joeynmt.training - Epoch  59: total training loss 162.82
2021-03-19 21:28:11,647 - INFO - joeynmt.training - EPOCH 60
2021-03-19 21:28:31,392 - INFO - joeynmt.training - Epoch  60: total training loss 162.17
2021-03-19 21:28:31,392 - INFO - joeynmt.training - EPOCH 61
2021-03-19 21:28:51,253 - INFO - joeynmt.training - Epoch  61: total training loss 161.85
2021

2021-03-19 21:35:52,725 - INFO - joeynmt.training - Epoch  82: total training loss 152.77
2021-03-19 21:35:52,725 - INFO - joeynmt.training - EPOCH 83
2021-03-19 21:36:14,263 - INFO - joeynmt.training - Epoch  83: total training loss 152.46
2021-03-19 21:36:14,263 - INFO - joeynmt.training - EPOCH 84
2021-03-19 21:36:35,719 - INFO - joeynmt.training - Epoch  84: total training loss 152.25
2021-03-19 21:36:35,720 - INFO - joeynmt.training - EPOCH 85
2021-03-19 21:36:57,473 - INFO - joeynmt.training - Epoch  85: total training loss 151.73
2021-03-19 21:36:57,473 - INFO - joeynmt.training - EPOCH 86
2021-03-19 21:37:18,230 - INFO - joeynmt.training - Epoch  86: total training loss 151.19
2021-03-19 21:37:18,230 - INFO - joeynmt.training - EPOCH 87
2021-03-19 21:37:39,248 - INFO - joeynmt.training - Epoch  87: total training loss 151.10
2021-03-19 21:37:39,249 - INFO - joeynmt.training - EPOCH 88
2021-03-19 21:38:00,208 - INFO - joeynmt.training - Epoch  88: total training loss 150.63
2021

## bpe 4000 con muchas capas

In [135]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves-sin2/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves-sin2"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [136]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 21:45:31,745 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 21:45:31,802 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 21:45:32,282 - INFO - joeynmt.data - building v

2021-03-19 21:46:33,212 - INFO - joeynmt.training - Epoch   1: total training loss 225.61
2021-03-19 21:46:33,212 - INFO - joeynmt.training - EPOCH 2
2021-03-19 21:47:26,426 - INFO - joeynmt.training - Epoch   2: total training loss 203.93
2021-03-19 21:47:26,427 - INFO - joeynmt.training - EPOCH 3
2021-03-19 21:48:19,619 - INFO - joeynmt.training - Epoch   3: total training loss 202.08
2021-03-19 21:48:19,619 - INFO - joeynmt.training - EPOCH 4
2021-03-19 21:49:03,839 - INFO - joeynmt.training - Epoch   4: total training loss 200.77
2021-03-19 21:49:03,840 - INFO - joeynmt.training - EPOCH 5
2021-03-19 21:49:54,526 - INFO - joeynmt.training - Epoch   5: total training loss 199.52
2021-03-19 21:49:54,527 - INFO - joeynmt.training - EPOCH 6
2021-03-19 21:50:45,992 - INFO - joeynmt.training - Epoch   6: total training loss 196.87
2021-03-19 21:50:45,992 - INFO - joeynmt.training - EPOCH 7
2021-03-19 21:51:35,728 - INFO - joeynmt.training - Epoch   7: total training loss 194.35
2021-03-19

2021-03-19 22:11:27,731 - INFO - joeynmt.training - Epoch  28: total training loss 149.63
2021-03-19 22:11:27,731 - INFO - joeynmt.training - EPOCH 29
2021-03-19 22:12:23,083 - INFO - joeynmt.training - Epoch  29: total training loss 147.76
2021-03-19 22:12:23,084 - INFO - joeynmt.training - EPOCH 30
2021-03-19 22:13:17,403 - INFO - joeynmt.training - Epoch  30: total training loss 145.45
2021-03-19 22:13:17,404 - INFO - joeynmt.training - EPOCH 31
2021-03-19 22:14:12,410 - INFO - joeynmt.training - Epoch  31: total training loss 143.66
2021-03-19 22:14:12,410 - INFO - joeynmt.training - EPOCH 32
2021-03-19 22:15:07,970 - INFO - joeynmt.training - Epoch  32: total training loss 142.01
2021-03-19 22:15:07,970 - INFO - joeynmt.training - EPOCH 33
2021-03-19 22:15:55,354 - INFO - joeynmt.training - Epoch  33: total training loss 140.97
2021-03-19 22:15:55,355 - INFO - joeynmt.training - EPOCH 34
2021-03-19 22:16:49,950 - INFO - joeynmt.training - Epoch  34: total training loss 140.92
2021

2021-03-19 22:36:17,369 - INFO - joeynmt.training - Epoch  55: total training loss 108.52
2021-03-19 22:36:17,369 - INFO - joeynmt.training - EPOCH 56
2021-03-19 22:37:04,170 - INFO - joeynmt.training - Epoch  56: total training loss 106.99
2021-03-19 22:37:04,171 - INFO - joeynmt.training - EPOCH 57
2021-03-19 22:37:56,337 - INFO - joeynmt.training - Epoch  57: total training loss 105.75
2021-03-19 22:37:56,337 - INFO - joeynmt.training - EPOCH 58
2021-03-19 22:38:49,443 - INFO - joeynmt.training - Epoch  58: total training loss 104.42
2021-03-19 22:38:49,443 - INFO - joeynmt.training - EPOCH 59
2021-03-19 22:39:39,822 - INFO - joeynmt.training - Epoch  59: total training loss 103.34
2021-03-19 22:39:39,822 - INFO - joeynmt.training - EPOCH 60
2021-03-19 22:40:31,770 - INFO - joeynmt.training - Epoch  60: total training loss 101.87
2021-03-19 22:40:31,770 - INFO - joeynmt.training - EPOCH 61
2021-03-19 22:41:23,792 - INFO - joeynmt.training - Epoch  61: total training loss 100.58
2021

2021-03-19 23:01:49,313 - INFO - joeynmt.training - Epoch  82: total training loss 78.39
2021-03-19 23:01:49,314 - INFO - joeynmt.training - EPOCH 83
2021-03-19 23:02:39,213 - INFO - joeynmt.training - Epoch  83: total training loss 77.49
2021-03-19 23:02:39,214 - INFO - joeynmt.training - EPOCH 84
2021-03-19 23:03:22,404 - INFO - joeynmt.training - Epoch  84: total training loss 76.39
2021-03-19 23:03:22,404 - INFO - joeynmt.training - EPOCH 85
2021-03-19 23:04:12,010 - INFO - joeynmt.training - Epoch  85: total training loss 75.95
2021-03-19 23:04:12,011 - INFO - joeynmt.training - EPOCH 86
2021-03-19 23:05:02,886 - INFO - joeynmt.training - Epoch  86: total training loss 74.91
2021-03-19 23:05:02,887 - INFO - joeynmt.training - EPOCH 87
2021-03-19 23:05:51,874 - INFO - joeynmt.training - Epoch  87: total training loss 74.22
2021-03-19 23:05:51,875 - INFO - joeynmt.training - EPOCH 88
2021-03-19 23:06:41,501 - INFO - joeynmt.training - Epoch  88: total training loss 73.24
2021-03-19 

### con 70 de longitud

In [137]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 70
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves-sin3/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves-sin3"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 70
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [138]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-19 23:28:08,122 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 23:28:08,214 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-19 23:28:08,719 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-19 23:29:26,897 - INFO - joeynmt.training - Epoch   1: total training loss 254.50
2021-03-19 23:29:26,898 - INFO - joeynmt.training - EPOCH 2
2021-03-19 23:30:39,083 - INFO - joeynmt.training - Epoch   2: total training loss 232.11
2021-03-19 23:30:39,083 - INFO - joeynmt.training - EPOCH 3
2021-03-19 23:31:42,006 - INFO - joeynmt.training - Epoch   3: total training loss 230.78
2021-03-19 23:31:42,006 - INFO - joeynmt.training - EPOCH 4
2021-03-19 23:32:51,023 - INFO - joeynmt.training - Epoch   4: total training loss 228.88
2021-03-19 23:32:51,023 - INFO - joeynmt.training - EP

2021-03-19 23:55:15,815 - INFO - joeynmt.training - Epoch  24: total training loss 173.13
2021-03-19 23:55:15,815 - INFO - joeynmt.training - EPOCH 25
2021-03-19 23:56:17,583 - INFO - joeynmt.training - Epoch  25: total training loss 170.96
2021-03-19 23:56:17,584 - INFO - joeynmt.training - EPOCH 26
2021-03-19 23:57:19,277 - INFO - joeynmt.training - Epoch  26: total training loss 168.58
2021-03-19 23:57:19,277 - INFO - joeynmt.training - EPOCH 27
2021-03-19 23:58:19,781 - INFO - joeynmt.training - Epoch  27: total training loss 166.78
2021-03-19 23:58:19,783 - INFO - joeynmt.training - EPOCH 28
2021-03-19 23:59:14,183 - INFO - joeynmt.training - Epoch  28: total training loss 164.46
2021-03-19 23:59:14,184 - INFO - joeynmt.training - EPOCH 29
2021-03-20 00:00:17,958 - INFO - joeynmt.training - Epoch  29: total training loss 162.50
2021-03-20 00:00:17,958 - INFO - joeynmt.training - EPOCH 30
2021-03-20 00:01:19,166 - INFO - joeynmt.training - Epoch  30: total training loss 161.80
2021

2021-03-20 00:21:23,712 - INFO - joeynmt.training - Epoch  48: total training loss 130.57
2021-03-20 00:21:23,712 - INFO - joeynmt.training - EPOCH 49
2021-03-20 00:22:25,729 - INFO - joeynmt.training - Epoch  49: total training loss 129.23
2021-03-20 00:22:25,729 - INFO - joeynmt.training - EPOCH 50
2021-03-20 00:23:28,198 - INFO - joeynmt.training - Epoch  50: total training loss 129.44
2021-03-20 00:23:28,198 - INFO - joeynmt.training - EPOCH 51
2021-03-20 00:24:22,911 - INFO - joeynmt.training - Epoch  51: total training loss 127.00
2021-03-20 00:24:22,911 - INFO - joeynmt.training - EPOCH 52
2021-03-20 00:25:26,363 - INFO - joeynmt.training - Epoch  52: total training loss 124.79
2021-03-20 00:25:26,363 - INFO - joeynmt.training - EPOCH 53
2021-03-20 00:26:26,609 - INFO - joeynmt.training - Epoch  53: total training loss 123.05
2021-03-20 00:26:26,609 - INFO - joeynmt.training - EPOCH 54
2021-03-20 00:27:27,823 - INFO - joeynmt.training - Epoch  54: total training loss 122.14
2021

2021-03-20 00:47:36,268 - INFO - joeynmt.training - Epoch  72: total training loss 100.18
2021-03-20 00:47:36,270 - INFO - joeynmt.training - EPOCH 73
2021-03-20 00:48:39,994 - INFO - joeynmt.training - Epoch  73: total training loss 98.94
2021-03-20 00:48:39,995 - INFO - joeynmt.training - EPOCH 74
2021-03-20 00:49:41,960 - INFO - joeynmt.training - Epoch  74: total training loss 97.92
2021-03-20 00:49:41,962 - INFO - joeynmt.training - EPOCH 75
2021-03-20 00:50:44,580 - INFO - joeynmt.training - Epoch  75: total training loss 97.21
2021-03-20 00:50:44,581 - INFO - joeynmt.training - EPOCH 76
2021-03-20 00:51:43,499 - INFO - joeynmt.training - Epoch  76: total training loss 95.98
2021-03-20 00:51:43,499 - INFO - joeynmt.training - EPOCH 77
2021-03-20 00:52:47,804 - INFO - joeynmt.training - Epoch  77: total training loss 95.40
2021-03-20 00:52:47,804 - INFO - joeynmt.training - EPOCH 78
2021-03-20 00:53:47,294 - INFO - joeynmt.training - Epoch  78: total training loss 94.27
2021-03-20

2021-03-20 01:14:45,209 - INFO - joeynmt.training - Epoch  96: total training loss 78.75
2021-03-20 01:14:45,209 - INFO - joeynmt.training - EPOCH 97
2021-03-20 01:15:51,161 - INFO - joeynmt.training - Epoch  97: total training loss 78.41
2021-03-20 01:15:51,161 - INFO - joeynmt.training - EPOCH 98
2021-03-20 01:16:50,423 - INFO - joeynmt.training - Epoch  98: total training loss 77.84
2021-03-20 01:16:50,425 - INFO - joeynmt.training - EPOCH 99
2021-03-20 01:17:55,231 - INFO - joeynmt.training - Epoch  99: total training loss 77.04
2021-03-20 01:17:55,232 - INFO - joeynmt.training - EPOCH 100
2021-03-20 01:18:57,296 - INFO - joeynmt.training - Epoch 100: total training loss 76.59
2021-03-20 01:18:57,298 - INFO - joeynmt.training - Training ended after 100 epochs.
2021-03-20 01:18:57,298 - INFO - joeynmt.training - Best validation result (greedy) at step     3000:  44.49 ppl.
2021-03-20 01:18:57,317 - INFO - joeynmt.prediction - Process device: cpu, n_gpu: 0, batch_size per device: 256

## con bpe 2000 pocas capas mir-es

In [147]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves-sin4/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves-sin4"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 3
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 64    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 64         # TODO: Increase to 512 for larger data.
        ff_size: 128            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [ ]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-20 01:28:47,313 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 01:28:47,372 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-20 01:28:48,010 - INFO - joeynmt.data - building v

/ve/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 01:29:03,452 - INFO - joeynmt.training - Epoch   1: total training loss 215.10
2021-03-20 01:29:03,454 - INFO - joeynmt.training - EPOCH 2
2021-03-20 01:29:16,062 - INFO - joeynmt.training - Epoch   2: total training loss 198.18
2021-03-20 01:29:16,063 - INFO - joeynmt.training - EPOCH 3
2021-03-20 01:29:28,591 - INFO - joeynmt.training - Epoch   3: total training loss 186.73
2021-03-20 01:29:28,592 - INFO - joeynmt.training - EPOCH 4
2021-03-20 01:29:40,770 - INFO - joeynmt.training - Epoch   4: total training loss 180.63
2021-03-20 01:29:40,771 - INFO - joeynmt.training - EP

2021-03-20 01:34:53,241 - INFO - joeynmt.training - Epoch  30: total training loss 157.88
2021-03-20 01:34:53,241 - INFO - joeynmt.training - EPOCH 31
2021-03-20 01:35:05,065 - INFO - joeynmt.training - Epoch  31: total training loss 157.20
2021-03-20 01:35:05,066 - INFO - joeynmt.training - EPOCH 32
2021-03-20 01:35:17,319 - INFO - joeynmt.training - Epoch  32: total training loss 156.64
2021-03-20 01:35:17,320 - INFO - joeynmt.training - EPOCH 33
2021-03-20 01:35:29,453 - INFO - joeynmt.training - Epoch  33: total training loss 155.91
2021-03-20 01:35:29,454 - INFO - joeynmt.training - EPOCH 34
2021-03-20 01:35:41,410 - INFO - joeynmt.training - Epoch  34: total training loss 155.28
2021-03-20 01:35:41,411 - INFO - joeynmt.training - EPOCH 35
2021-03-20 01:35:53,027 - INFO - joeynmt.training - Epoch  35: total training loss 154.74
2021-03-20 01:35:53,028 - INFO - joeynmt.training - EPOCH 36
2021-03-20 01:36:05,087 - INFO - joeynmt.training - Epoch  36: total training loss 154.07
2021

### Con bpe 2000 muchas capas sin random mir-es

In [154]:
# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{data_path}/train.bpe"
    dev:   "{data_path}/dev.bpe"
    test:  "{data_path}/test.bpe"
    level: "bpe"
    lowercase: true
    max_sent_length: 50
    src_vocab: "{data_path}/vocab.txt"
    trg_vocab: "{data_path}/vocab.txt"
    
testing:                            # specify which inference algorithm to use for testing (for validation it's always greedy decoding)
    beam_size: 3                    # size of the beam for beam search
    alpha: 1.0                      # length penalty for beam search
    postprocess: True               # remove BPE splits after translation, this only applies if "level" is "bpe" (default: True)
    bpe_type: "subword-nmt"         # bpe type: either "subword-nmt" or "sentencepiece" (default: subword-nmt; used in postprocess during evaluation)
    sacrebleu:                      # sacrebleu options
        remove_whitespace: True     # `remove_whitespace` option in sacrebleu.corpus_chrf() function (defalut: True)
        tokenize: "13a"             # `tokenize` option in sacrebleu.corpus_bleu() function (default: 13a)

training:
    #load_model: "models/alreves-sin5/.kpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 800     # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 128
    batch_type: "sentence"
    eval_batch_size: 256
    eval_batch_type: "sentence"
    batch_multiplier: 1
    early_stopping_metric: "ppl"   #perplejidad
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 500          # TODO: Set to at least once per epoch.    #cada 500 se hace la validacion
    logging_freq: 1000            
    eval_metric: "bleu"
    model_dir: "models/alreves-sin5"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 50
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(data_path=data_path, name=name,source_language=source_language, target_language=target_language)
with codecs.open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w',encoding='utf-8') as f:
    f.write(config)

In [155]:
! cd joeynmt; /ve/bin/python3 -m joeynmt train configs/transformer_{name}.yaml

2021-03-20 02:29:03,206 - INFO - root - Hello! This is Joey-NMT (version 1.0).
/ve/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
2021-03-20 02:29:03,318 - INFO - joeynmt.data - loading training data...
/ve/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
2021-03-20 02:29:03,661 - INFO - joeynmt.data - building v

2021-03-20 02:29:58,384 - INFO - joeynmt.training - Epoch   1: total training loss 192.55
2021-03-20 02:29:58,384 - INFO - joeynmt.training - EPOCH 2
2021-03-20 02:30:34,719 - INFO - joeynmt.training - Epoch   2: total training loss 176.82
2021-03-20 02:30:34,721 - INFO - joeynmt.training - EPOCH 3
2021-03-20 02:31:15,435 - INFO - joeynmt.training - Epoch   3: total training loss 175.35
2021-03-20 02:31:15,437 - INFO - joeynmt.training - EPOCH 4
2021-03-20 02:31:57,773 - INFO - joeynmt.training - Epoch   4: total training loss 174.37
2021-03-20 02:31:57,773 - INFO - joeynmt.training - EPOCH 5
2021-03-20 02:32:39,729 - INFO - joeynmt.training - Epoch   5: total training loss 173.11
2021-03-20 02:32:39,729 - INFO - joeynmt.training - EPOCH 6
2021-03-20 02:33:19,619 - INFO - joeynmt.training - Epoch   6: total training loss 171.06
2021-03-20 02:33:19,620 - INFO - joeynmt.training - EPOCH 7
2021-03-20 02:34:00,627 - INFO - joeynmt.training - Epoch   7: total training loss 170.23
2021-03-20

2021-03-20 02:51:19,841 - INFO - joeynmt.training - Epoch  30: total training loss 117.28
2021-03-20 02:51:19,842 - INFO - joeynmt.training - EPOCH 31
2021-03-20 02:52:01,758 - INFO - joeynmt.training - Epoch  31: total training loss 115.71
2021-03-20 02:52:01,762 - INFO - joeynmt.training - EPOCH 32
2021-03-20 02:52:43,395 - INFO - joeynmt.training - Epoch  32: total training loss 114.64
2021-03-20 02:52:43,396 - INFO - joeynmt.training - EPOCH 33
2021-03-20 02:53:22,972 - INFO - joeynmt.training - Epoch  33: total training loss 112.97
2021-03-20 02:53:22,974 - INFO - joeynmt.training - EPOCH 34
2021-03-20 02:53:59,763 - INFO - joeynmt.training - Epoch  34: total training loss 111.07
2021-03-20 02:53:59,764 - INFO - joeynmt.training - EPOCH 35
2021-03-20 02:54:40,760 - INFO - joeynmt.training - Epoch  35: total training loss 110.01
2021-03-20 02:54:40,762 - INFO - joeynmt.training - EPOCH 36
2021-03-20 02:55:22,772 - INFO - joeynmt.training - Epoch  36: total training loss 108.55
2021

2021-03-20 03:13:21,614 - INFO - joeynmt.training - Epoch  59: total training loss 84.30
2021-03-20 03:13:21,614 - INFO - joeynmt.training - EPOCH 60
2021-03-20 03:14:05,776 - INFO - joeynmt.training - Epoch  60: total training loss 83.56
2021-03-20 03:14:05,776 - INFO - joeynmt.training - EPOCH 61
2021-03-20 03:14:41,067 - INFO - joeynmt.training - Epoch  61: total training loss 82.52
2021-03-20 03:14:41,068 - INFO - joeynmt.training - EPOCH 62
2021-03-20 03:15:25,225 - INFO - joeynmt.training - Epoch  62: total training loss 81.69
2021-03-20 03:15:25,225 - INFO - joeynmt.training - EPOCH 63
2021-03-20 03:16:09,850 - INFO - joeynmt.training - Epoch  63: total training loss 81.44
2021-03-20 03:16:09,850 - INFO - joeynmt.training - EPOCH 64
2021-03-20 03:16:53,463 - INFO - joeynmt.training - Epoch  64: total training loss 80.34
2021-03-20 03:16:53,463 - INFO - joeynmt.training - EPOCH 65
2021-03-20 03:17:29,320 - INFO - joeynmt.training - Epoch  65: total training loss 79.22
2021-03-20 

2021-03-20 03:34:58,539 - INFO - joeynmt.training - Epoch  89: total training loss 62.68
2021-03-20 03:34:58,539 - INFO - joeynmt.training - EPOCH 90
2021-03-20 03:35:31,742 - INFO - joeynmt.training - Epoch  90: total training loss 62.40
2021-03-20 03:35:31,742 - INFO - joeynmt.training - EPOCH 91
2021-03-20 03:36:11,205 - INFO - joeynmt.training - Epoch  91: total training loss 61.70
2021-03-20 03:36:11,205 - INFO - joeynmt.training - EPOCH 92
2021-03-20 03:36:49,785 - INFO - joeynmt.training - Epoch  92: total training loss 61.27
2021-03-20 03:36:49,786 - INFO - joeynmt.training - EPOCH 93
2021-03-20 03:37:28,469 - INFO - joeynmt.training - Epoch  93: total training loss 61.12
2021-03-20 03:37:28,469 - INFO - joeynmt.training - EPOCH 94
2021-03-20 03:38:07,074 - INFO - joeynmt.training - Epoch  94: total training loss 60.12
2021-03-20 03:38:07,074 - INFO - joeynmt.training - EPOCH 95
2021-03-20 03:38:45,758 - INFO - joeynmt.training - Epoch  95: total training loss 59.75
2021-03-20 